In [21]:
from xgboost import XGBRegressor

import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 200)

import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'svg'

from tqdm import tqdm

from statsmodels.tsa.tsatools import lagmat

import matplotlib.pylab as plt
%matplotlib inline

In [3]:
transactions = pd.read_csv('transactions.csv')
transactions['day'] = transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

transactions['pos_amount'] = transactions.amount.apply(lambda x: 0 if x<0 else np.log(x + 1))
transactions['neg_amount'] = transactions.amount.apply(lambda x: 0 if x>0 else -x)

transactions.drop(['amount', 'term_id', 'tr_datetime'], 1, inplace=True)

test_transactions = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                                 index=np.arange(1, 31) + transactions.day.max())
test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)
test_transactions.columns = ['mcc_code', 'day']


train_grid = pd.DataFrame(columns=transactions.mcc_code.unique(), 
                          index=transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

for tr_table in tqdm([transactions, test_transactions, train_grid]):
    tr_table['week_num'] = (tr_table['day'] - 2) // 7
    tr_table['week_day'] = (tr_table['day'] - 2) % 7
    tr_table['month_num'] = tr_table['day'] // 28
    tr_table['month_day'] = tr_table['day'] % 28
    
merge_col_names = ['day', 'week_num', 'week_day', 'month_num', 'month_day', 'mcc_code']

train_transactions = pd.merge(
    train_grid,
    transactions.groupby(merge_col_names)[['neg_amount', 'pos_amount']].sum().reset_index(),
    how='left').fillna(0)

train_transactions = pd.merge(
    train_transactions,
    transactions.groupby(merge_col_names)[['customer_id']].count().reset_index(),
    how='left').fillna(0).astype(np.int32)
train_transactions.columns = np.hstack([train_transactions.columns[:-1], ['n_transactions']])

train_transactions['log_neg_amount'] = train_transactions.neg_amount.apply(lambda x: np.log(x + 1))
train_transactions['log_pos_amount'] = train_transactions.pos_amount.apply(lambda x: np.log(x + 1))
train_transactions.head()

100%|██████████| 3/3 [00:00<00:00,  2.04it/s]


,mcc_code,day,week_num,week_day,month_num,month_day,neg_amount,pos_amount,n_transactions,log_neg_amount,log_pos_amount
0,4814,0,-1,5,0,0,11098744,0,2365,16.222343,0.0
1,4814,1,-1,6,0,1,7881825,0,1697,15.880070,0.0
2,4814,2,0,0,0,2,6777480,0,1524,15.729116,0.0
3,4814,3,0,1,0,3,9277943,0,1937,16.043151,0.0
4,4814,4,0,2,0,4,9999757,0,1943,16.118071,0.0


In [4]:
train = train_transactions.copy()

helper = train.copy().set_index(['mcc_code', 'week_num'])
helper['week_mean'] = train_transactions.groupby(['mcc_code', 'week_num']).mean()['log_neg_amount']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'week_num'])
helper['week_std'] = train_transactions.groupby(['mcc_code', 'week_num']).std()['log_neg_amount']
train = helper.reset_index().copy()
train.head()

helper = train.copy().set_index(['mcc_code', 'week_num'])
helper['week_mean_transactions'] = train_transactions.groupby(['mcc_code', 'week_num']).mean()['n_transactions']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'week_num'])
helper['week_std_transactions'] = train_transactions.groupby(['mcc_code', 'week_num']).std()['n_transactions']
train = helper.reset_index().copy()
train.head()

,mcc_code,week_num,day,week_day,month_num,month_day,neg_amount,pos_amount,n_transactions,log_neg_amount,log_pos_amount,week_mean,week_std,week_mean_transactions,week_std_transactions
0,4814,-1,0,5,0,0,11098744,0,2365,16.222343,0.0,16.051206,0.242023,2031.000000,472.347330
1,4814,-1,1,6,0,1,7881825,0,1697,15.880070,0.0,16.051206,0.242023,2031.000000,472.347330
2,4814,0,2,0,0,2,6777480,0,1524,15.729116,0.0,15.985827,0.141873,1821.571429,186.298915
3,4814,0,3,1,0,3,9277943,0,1937,16.043151,0.0,15.985827,0.141873,1821.571429,186.298915
4,4814,0,4,2,0,4,9999757,0,1943,16.118071,0.0,15.985827,0.141873,1821.571429,186.298915


In [5]:
helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_mean'] = train_transactions.groupby(['mcc_code', 'month_num']).mean()['log_neg_amount']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_std'] = train_transactions.groupby(['mcc_code', 'month_num']).std()['log_neg_amount']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_median'] = train_transactions.groupby(['mcc_code', 'month_num']).median()['log_neg_amount']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_min'] = train_transactions.groupby(['mcc_code', 'month_num']).min()['log_neg_amount']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_max'] = train_transactions.groupby(['mcc_code', 'month_num']).max()['log_neg_amount']
train = helper.reset_index().copy()

#n_transactions
helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_mean_transactions'] = train_transactions.groupby(['mcc_code', 'month_num']).mean()['n_transactions']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_std_transactions'] = train_transactions.groupby(['mcc_code', 'month_num']).std()['n_transactions']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_median_transactions'] = train_transactions.groupby(['mcc_code', 'month_num']).median()['n_transactions']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_min_transactions'] = train_transactions.groupby(['mcc_code', 'month_num']).min()['n_transactions']
train = helper.reset_index().copy()

helper = train.copy().set_index(['mcc_code', 'month_num'])
helper['month_max_transactions'] = train_transactions.groupby(['mcc_code', 'month_num']).max()['n_transactions']
train = helper.reset_index().copy()

TIMESTAT_COLS = train.columns.difference(train_transactions.columns)

train.head()

,mcc_code,month_num,week_num,day,week_day,month_day,neg_amount,pos_amount,n_transactions,log_neg_amount,log_pos_amount,week_mean,week_std,week_mean_transactions,week_std_transactions,month_mean,month_std,month_median,month_min,month_max,month_mean_transactions,month_std_transactions,month_median_transactions,month_min_transactions,month_max_transactions
0,4814,0,-1,0,5,0,11098744,0,2365,16.222343,0.0,16.051206,0.242023,2031.000000,472.347330,16.024313,0.164409,16.082561,15.694662,16.227089,1921.607143,230.296321,1986.5,1513,2365
1,4814,0,-1,1,6,1,7881825,0,1697,15.880070,0.0,16.051206,0.242023,2031.000000,472.347330,16.024313,0.164409,16.082561,15.694662,16.227089,1921.607143,230.296321,1986.5,1513,2365
2,4814,0,0,2,0,2,6777480,0,1524,15.729116,0.0,15.985827,0.141873,1821.571429,186.298915,16.024313,0.164409,16.082561,15.694662,16.227089,1921.607143,230.296321,1986.5,1513,2365
3,4814,0,0,3,1,3,9277943,0,1937,16.043151,0.0,15.985827,0.141873,1821.571429,186.298915,16.024313,0.164409,16.082561,15.694662,16.227089,1921.607143,230.296321,1986.5,1513,2365
4,4814,0,0,4,2,4,9999757,0,1943,16.118071,0.0,15.985827,0.141873,1821.571429,186.298915,16.024313,0.164409,16.082561,15.694662,16.227089,1921.607143,230.296321,1986.5,1513,2365


In [6]:
week_lag_max = 35
# week_pos_lag_max = 20
for week_shift in tqdm(np.arange(1, week_lag_max)):
    train_shift = train.copy()
    train_shift['week_num'] += week_shift
    train_shift['prev_week_{}_neg'.format(week_shift)] = train_shift.log_neg_amount
    train_shift['prev_week_{}_mean_neg'.format(week_shift)] = train_shift.week_mean
    train_shift['prev_week_{}_std_neg'.format(week_shift)] = train_shift.week_std
    
    #n_transactions
    train_shift['prev_week_{}_n_trans'.format(week_shift)] = train_shift.n_transactions
    train_shift['prev_week_{}_mean_n_trans'.format(week_shift)] = train_shift.week_mean_transactions
    train_shift['prev_week_{}_std_n_trans'.format(week_shift)] = train_shift.week_std_transactions
    
    train_shift = train_shift[[
        'week_num', 'week_day', 'mcc_code',
        'prev_week_{}_neg'.format(week_shift),
        'prev_week_{}_mean_neg'.format(week_shift),
        'prev_week_{}_std_neg'.format(week_shift),
        #added
        'prev_week_{}_n_trans'.format(week_shift),
        'prev_week_{}_mean_n_trans'.format(week_shift),
        'prev_week_{}_std_n_trans'.format(week_shift),
    ]]
    train_transactions = pd.merge(
        train_transactions, train_shift, 
        on=['week_num', 'week_day', 'mcc_code'],
        how='left').fillna(0)
    test_transactions = pd.merge(
        test_transactions, train_shift, 
        on=['week_num', 'week_day', 'mcc_code'],
        how='left').fillna(0)
train_transactions.head()

100%|██████████| 34/34 [00:07<00:00,  2.71it/s]


,mcc_code,day,week_num,week_day,month_num,month_day,neg_amount,pos_amount,n_transactions,log_neg_amount,log_pos_amount,prev_week_1_neg,prev_week_1_mean_neg,prev_week_1_std_neg,prev_week_1_n_trans,prev_week_1_mean_n_trans,prev_week_1_std_n_trans,prev_week_2_neg,prev_week_2_mean_neg,prev_week_2_std_neg,prev_week_2_n_trans,prev_week_2_mean_n_trans,prev_week_2_std_n_trans,prev_week_3_neg,prev_week_3_mean_neg,prev_week_3_std_neg,prev_week_3_n_trans,prev_week_3_mean_n_trans,prev_week_3_std_n_trans,prev_week_4_neg,prev_week_4_mean_neg,prev_week_4_std_neg,prev_week_4_n_trans,prev_week_4_mean_n_trans,prev_week_4_std_n_trans,prev_week_5_neg,prev_week_5_mean_neg,prev_week_5_std_neg,prev_week_5_n_trans,prev_week_5_mean_n_trans,prev_week_5_std_n_trans,prev_week_6_neg,prev_week_6_mean_neg,prev_week_6_std_neg,prev_week_6_n_trans,prev_week_6_mean_n_trans,prev_week_6_std_n_trans,prev_week_7_neg,prev_week_7_mean_neg,prev_week_7_std_neg,prev_week_7_n_trans,prev_week_7_mean_n_trans,prev_week_7_std_n_trans,prev_week_8_neg,prev_week_8_mean_neg,prev_week_8_std_neg,prev_week_8_n_trans,prev_week_8_mean_n_trans,prev_week_8_std_n_trans,prev_week_9_neg,prev_week_9_mean_neg,prev_week_9_std_neg,prev_week_9_n_trans,prev_week_9_mean_n_trans,prev_week_9_std_n_trans,prev_week_10_neg,prev_week_10_mean_neg,prev_week_10_std_neg,prev_week_10_n_trans,prev_week_10_mean_n_trans,prev_week_10_std_n_trans,prev_week_11_neg,prev_week_11_mean_neg,prev_week_11_std_neg,prev_week_11_n_trans,prev_week_11_mean_n_trans,prev_week_11_std_n_trans,prev_week_12_neg,prev_week_12_mean_neg,prev_week_12_std_neg,prev_week_12_n_trans,prev_week_12_mean_n_trans,prev_week_12_std_n_trans,prev_week_13_neg,prev_week_13_mean_neg,prev_week_13_std_neg,prev_week_13_n_trans,prev_week_13_mean_n_trans,prev_week_13_std_n_trans,prev_week_14_neg,prev_week_14_mean_neg,prev_week_14_std_neg,prev_week_14_n_trans,prev_week_14_mean_n_trans,prev_week_14_std_n_trans,prev_week_15_neg,prev_week_15_mean_neg,prev_week_15_std_neg,prev_week_15_n_trans,prev_week_15_mean_n_trans,...,prev_week_18_std_neg,prev_week_18_n_trans,prev_week_18_mean_n_trans,prev_week_18_std_n_trans,prev_week_19_neg,prev_week_19_mean_neg,prev_week_19_std_neg,prev_week_19_n_trans,prev_week_19_mean_n_trans,prev_week_19_std_n_trans,prev_week_20_neg,prev_week_20_mean_neg,prev_week_20_std_neg,prev_week_20_n_trans,prev_week_20_mean_n_trans,prev_week_20_std_n_trans,prev_week_21_neg,prev_week_21_mean_neg,prev_week_21_std_neg,prev_week_21_n_trans,prev_week_21_mean_n_trans,prev_week_21_std_n_trans,prev_week_22_neg,prev_week_22_mean_neg,prev_week_22_std_neg,prev_week_22_n_trans,prev_week_22_mean_n_trans,prev_week_22_std_n_trans,prev_week_23_neg,prev_week_23_mean_neg,prev_week_23_std_neg,prev_week_23_n_trans,prev_week_23_mean_n_trans,prev_week_23_std_n_trans,prev_week_24_neg,prev_week_24_mean_neg,prev_week_24_std_neg,prev_week_24_n_trans,prev_week_24_mean_n_trans,prev_week_24_std_n_trans,prev_week_25_neg,prev_week_25_mean_neg,prev_week_25_std_neg,prev_week_25_n_trans,prev_week_25_mean_n_trans,prev_week_25_std_n_trans,prev_week_26_neg,prev_week_26_mean_neg,prev_week_26_std_neg,prev_week_26_n_trans,prev_week_26_mean_n_trans,prev_week_26_std_n_trans,prev_week_27_neg,prev_week_27_mean_neg,prev_week_27_std_neg,prev_week_27_n_trans,prev_week_27_mean_n_trans,prev_week_27_std_n_trans,prev_week_28_neg,prev_week_28_mean_neg,prev_week_28_std_neg,prev_week_28_n_trans,prev_week_28_mean_n_trans,prev_week_28_std_n_trans,prev_week_29_neg,prev_week_29_mean_neg,prev_week_29_std_neg,prev_week_29_n_trans,prev_week_29_mean_n_trans,prev_week_29_std_n_trans,prev_week_30_neg,prev_week_30_mean_neg,prev_week_30_std_neg,prev_week_30_n_trans,prev_week_30_mean_n_trans,prev_week_30_std_n_trans,prev_week_31_neg,prev_week_31_mean_neg,prev_week_31_std_neg,prev_week_31_n_trans,prev_week_31_mean_n_trans,prev_week_31_std_n_trans,prev_week_32_neg,prev_week_32_mean_neg,prev_week_32_std_neg,prev_week_32_n_trans,prev_week_32_mean_n_trans,prev_week_32_std_n_trans,prev_week_33_neg,prev_week_33_mean_

In [7]:
month_lag_max = 10
for month_shift in tqdm(np.arange(1, month_lag_max)):
    train_shift = train.copy()
    train_shift['month_num'] += month_shift
    train_shift['prev_month_{}_mean_neg'.format(month_shift)] = train_shift.month_mean
    train_shift['prev_month_{}_std_neg'.format(month_shift)] = train_shift.month_std
    train_shift['prev_month_{}_median_neg'.format(month_shift)] = train_shift.month_median
    train_shift['prev_month_{}_max_neg'.format(month_shift)] = train_shift.month_max
    train_shift['prev_month_{}_min_neg'.format(month_shift)] = train_shift.month_min
    
    #added
    train_shift['prev_month_{}_mean_n_trans'.format(month_shift)] = train_shift.month_mean_transactions
    train_shift['prev_month_{}_std_n_trans'.format(month_shift)] = train_shift.month_std_transactions
    train_shift['prev_month_{}_median_n_trans'.format(month_shift)] = train_shift.month_median_transactions
    train_shift['prev_month_{}_max_n_trans'.format(month_shift)] = train_shift.month_max_transactions
    train_shift['prev_month_{}_min_n_trans'.format(month_shift)] = train_shift.month_min_transactions
    
    train_shift = train_shift[[
        'month_num', 'day', 'mcc_code',
        'prev_month_{}_mean_neg'.format(month_shift),
        'prev_month_{}_std_neg'.format(month_shift),
        'prev_month_{}_median_neg'.format(month_shift),
        'prev_month_{}_max_neg'.format(month_shift),
        'prev_month_{}_min_neg'.format(month_shift),
        'prev_month_{}_mean_n_trans'.format(month_shift),
        'prev_month_{}_std_n_trans'.format(month_shift),
        'prev_month_{}_median_n_trans'.format(month_shift),
        'prev_month_{}_max_n_trans'.format(month_shift),
        'prev_month_{}_min_n_trans'.format(month_shift),
    ]]
    train_transactions = pd.merge(
        train_transactions, train_shift, 
        on=['month_num', 'day', 'mcc_code'],
        how='left').fillna(0)
    test_transactions = pd.merge(
        test_transactions, train_shift, 
        on=['month_num', 'day', 'mcc_code'],
        how='left').fillna(0)
train_transactions.head()

100%|██████████| 9/9 [00:04<00:00,  1.96it/s]


,mcc_code,day,week_num,week_day,month_num,month_day,neg_amount,pos_amount,n_transactions,log_neg_amount,log_pos_amount,prev_week_1_neg,prev_week_1_mean_neg,prev_week_1_std_neg,prev_week_1_n_trans,prev_week_1_mean_n_trans,prev_week_1_std_n_trans,prev_week_2_neg,prev_week_2_mean_neg,prev_week_2_std_neg,prev_week_2_n_trans,prev_week_2_mean_n_trans,prev_week_2_std_n_trans,prev_week_3_neg,prev_week_3_mean_neg,prev_week_3_std_neg,prev_week_3_n_trans,prev_week_3_mean_n_trans,prev_week_3_std_n_trans,prev_week_4_neg,prev_week_4_mean_neg,prev_week_4_std_neg,prev_week_4_n_trans,prev_week_4_mean_n_trans,prev_week_4_std_n_trans,prev_week_5_neg,prev_week_5_mean_neg,prev_week_5_std_neg,prev_week_5_n_trans,prev_week_5_mean_n_trans,prev_week_5_std_n_trans,prev_week_6_neg,prev_week_6_mean_neg,prev_week_6_std_neg,prev_week_6_n_trans,prev_week_6_mean_n_trans,prev_week_6_std_n_trans,prev_week_7_neg,prev_week_7_mean_neg,prev_week_7_std_neg,prev_week_7_n_trans,prev_week_7_mean_n_trans,prev_week_7_std_n_trans,prev_week_8_neg,prev_week_8_mean_neg,prev_week_8_std_neg,prev_week_8_n_trans,prev_week_8_mean_n_trans,prev_week_8_std_n_trans,prev_week_9_neg,prev_week_9_mean_neg,prev_week_9_std_neg,prev_week_9_n_trans,prev_week_9_mean_n_trans,prev_week_9_std_n_trans,prev_week_10_neg,prev_week_10_mean_neg,prev_week_10_std_neg,prev_week_10_n_trans,prev_week_10_mean_n_trans,prev_week_10_std_n_trans,prev_week_11_neg,prev_week_11_mean_neg,prev_week_11_std_neg,prev_week_11_n_trans,prev_week_11_mean_n_trans,prev_week_11_std_n_trans,prev_week_12_neg,prev_week_12_mean_neg,prev_week_12_std_neg,prev_week_12_n_trans,prev_week_12_mean_n_trans,prev_week_12_std_n_trans,prev_week_13_neg,prev_week_13_mean_neg,prev_week_13_std_neg,prev_week_13_n_trans,prev_week_13_mean_n_trans,prev_week_13_std_n_trans,prev_week_14_neg,prev_week_14_mean_neg,prev_week_14_std_neg,prev_week_14_n_trans,prev_week_14_mean_n_trans,prev_week_14_std_n_trans,prev_week_15_neg,prev_week_15_mean_neg,prev_week_15_std_neg,prev_week_15_n_trans,prev_week_15_mean_n_trans,...,prev_week_33_std_neg,prev_week_33_n_trans,prev_week_33_mean_n_trans,prev_week_33_std_n_trans,prev_week_34_neg,prev_week_34_mean_neg,prev_week_34_std_neg,prev_week_34_n_trans,prev_week_34_mean_n_trans,prev_week_34_std_n_trans,prev_month_1_mean_neg,prev_month_1_std_neg,prev_month_1_median_neg,prev_month_1_max_neg,prev_month_1_min_neg,prev_month_1_mean_n_trans,prev_month_1_std_n_trans,prev_month_1_median_n_trans,prev_month_1_max_n_trans,prev_month_1_min_n_trans,prev_month_2_mean_neg,prev_month_2_std_neg,prev_month_2_median_neg,prev_month_2_max_neg,prev_month_2_min_neg,prev_month_2_mean_n_trans,prev_month_2_std_n_trans,prev_month_2_median_n_trans,prev_month_2_max_n_trans,prev_month_2_min_n_trans,prev_month_3_mean_neg,prev_month_3_std_neg,prev_month_3_median_neg,prev_month_3_max_neg,prev_month_3_min_neg,prev_month_3_mean_n_trans,prev_month_3_std_n_trans,prev_month_3_median_n_trans,prev_month_3_max_n_trans,prev_month_3_min_n_trans,prev_month_4_mean_neg,prev_month_4_std_neg,prev_month_4_median_neg,prev_month_4_max_neg,prev_month_4_min_neg,prev_month_4_mean_n_trans,prev_month_4_std_n_trans,prev_month_4_median_n_trans,prev_month_4_max_n_trans,prev_month_4_min_n_trans,prev_month_5_mean_neg,prev_month_5_std_neg,prev_month_5_median_neg,prev_month_5_max_neg,prev_month_5_min_neg,prev_month_5_mean_n_trans,prev_month_5_std_n_trans,prev_month_5_median_n_trans,prev_month_5_max_n_trans,prev_month_5_min_n_trans,prev_month_6_mean_neg,prev_month_6_std_neg,prev_month_6_median_neg,prev_month_6_max_neg,prev_month_6_min_neg,prev_month_6_mean_n_trans,prev_month_6_std_n_trans,prev_month_6_median_n_trans,prev_month_6_max_n_trans,prev_month_6_min_n_trans,prev_month_7_mean_neg,prev_month_7_std_neg,prev_month_7_median_neg,prev_month_7_max_neg,prev_month_7_min_neg,prev_month_7_mean_n_trans,prev_month_7_std_n_trans,prev_month_7_median_n_trans,prev_month_7_max_n_trans,prev_month_7_min_n_trans,prev_month_8_mean_neg,prev_month_8_std_neg,prev_month_8_median_neg,prev_month_8_max_n

In [8]:
dummy_train = pd.get_dummies(train_transactions, columns=['mcc_code'])
dummy_test = pd.get_dummies(test_transactions, columns=['mcc_code'])

def rmsle(predicted, actual):
    assert(len(predicted) == len(actual))
    p = np.log(np.array(predicted) + 1)
    a = np.log(np.array(actual) + 1)
    return (((p - a)**2).sum() / len(predicted))**0.5

def rmsle_by_logs(predicted, actual):
    assert(len(predicted) == len(actual))
    return (((predicted - actual)**2).sum() / len(predicted))**0.5

def eval_model(labeled_data, target_col_name, clf, day_shifts=np.arange(90, 0, -15)):
    max_day = labeled_data.day.max()
    c = labeled_data.columns.difference([target_col_name])
    metric_by_shift = {}
    for day_shift in tqdm(day_shifts): 
        train_sample = labeled_data[labeled_data.day <= max_day - day_shift]
        test_sample = labeled_data[labeled_data.day > max_day - day_shift]
        clf.fit(train_sample[c], train_sample[target_col_name])
        predicted_volume = clf.predict(test_sample[c])
        metric_by_shift['RMSLE with {} days'.format(day_shift)] = rmsle_by_logs(
            predicted_volume, test_sample[target_col_name])
    return metric_by_shift

def test_weeks_preparation(X_test):
    X_test_by_weeks = []
    current_test_week = X_test[X_test.week_num == 65].copy()
    X_test_by_weeks.append(current_test_week)
    removing_month_cols = ['prev_month_1_mean_neg', 'prev_month_1_std_neg', 'prev_month_1_median_neg', 
                           'prev_month_1_max_neg', 'prev_month_1_min_neg',
#                            'prev_month_1_mean_pos', 'prev_month_1_std_pos', 'prev_month_1_median_pos'
                          ]
    removing_week_cols = []
    for prev_week_index in range(1, 5):
        current_test_week = X_test[X_test.week_num == 65 + prev_week_index].copy()
        current_test_week.drop(removing_month_cols, 1, inplace=True)
        removing_week_cols += [
            'prev_week_{}_neg'.format(prev_week_index),
            'prev_week_{}_mean_neg'.format(prev_week_index),
            'prev_week_{}_std_neg'.format(prev_week_index),
            'prev_week_{}_n_trans'.format(prev_week_index),
            'prev_week_{}_mean_n_trans'.format(prev_week_index),
            'prev_week_{}_std_n_trans'.format(prev_week_index),
            
        ]
        current_test_week.drop(removing_week_cols, 1, inplace=True)
        X_test_by_weeks.append(current_test_week)
    return X_test_by_weeks

def eval_nestimators(X, y, days_for_test, clf):
    max_day = X.day.max()
    X_train = X[X.day <= max_day - days_for_test]
    y_train = y[X.day <= max_day - days_for_test]
    X_test = X[X.day > max_day - days_for_test]
    y_test = y[X.day > max_day - days_for_test]
    
    eval_set  = [(X_train, y_train), (X_test, y_test)]
    clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='rmse', eval_set=eval_set)
    return clf.best_iteration

def get_agile_prediction(X_train, y_train, X_test, clf, test_form):
    X_test_by_weeks = test_weeks_preparation(X_test)
    out_form = pd.DataFrame()
    for prev_week_index in tqdm(range(5)):
        clf.n_estimators = 3000
        current_test = X_test_by_weeks[prev_week_index] 
        drop_cols = X_train.columns.difference(current_test.columns)
        current_train = X_train.drop(drop_cols, 1)
    
        clf.n_estimators = eval_nestimators(current_train, y_train, 30, clf)
        print(clf.n_estimators)
        clf.fit(current_train, y_train)
        predicted_log_volume = clf.predict(current_test)
        
        current_out_form = test_form[test_form.week_num == 65 + prev_week_index].copy()
        current_out_form['id'] = current_out_form[['mcc_code', 'day']].apply(
            lambda x: '-'.join(map(str, x)), axis=1)
        current_out_form['volume'] = np.expm1(predicted_log_volume)
        out_form = out_form.append(current_out_form[['id', 'volume']])
    return out_form

In [14]:
from xgboost import XGBRegressor
clf = XGBRegressor(learning_rate=0.02, max_depth=6, n_estimators=350, nthread=-1, reg_alpha=1, reg_lambda=1)
drop_cols = [
    'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]
metric_by_shift = eval_model(
    dummy_train.drop(drop_cols, 1),
    'log_neg_amount', clf, [60, 30, 14, 7])
for elem in metric_by_shift:
    print('{0}: {1}'.format(elem, metric_by_shift[elem]))


100%|██████████| 4/4 [05:49<00:00, 87.29s/it]

RMSLE with 60 days: 3.3237648930772425
RMSLE with 30 days: 3.3049292712598937
RMSLE with 14 days: 3.3267559156817503
RMSLE with 7 days: 3.2978361942636503


In [ ]:
from xgboost import XGBRegressor
clf = XGBRegressor(learning_rate=0.02, max_depth=6, n_estimators=350, nthread=-1, reg_alpha=1, reg_lambda=1)
drop_cols = [
    'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]
metric_by_shift = eval_model(
    dummy_train.drop(drop_cols, 1),
    'log_neg_amount', clf, [60, 30, 14, 7])
for elem in metric_by_shift:
    print('{0}: {1}'.format(elem, metric_by_shift[elem]))

In [30]:
from catboost import CatBoostRegressor

def eval_model(labeled_data, target_col_name, clf, day_shifts=np.arange(90, 0, -15)):
    
    max_day = labeled_data.day.max()
    c = labeled_data.columns.difference([target_col_name])
    metric_by_shift = {}
    for day_shift in tqdm(day_shifts): 
        train_sample = labeled_data[labeled_data.day <= max_day - day_shift]
        test_sample = labeled_data[labeled_data.day > max_day - day_shift]
        
        clf = CatBoostRegressor(
            use_best_model=True, eval_metric='RMSE',
            eval_set=((test_sample[c], test_sample[target_col_name])), random_seed=42)
        clf.fit(train_sample[c], train_sample[target_col_name])
        predicted_volume = clf.predict(test_sample[c])
        metric_by_shift['RMSLE with {} days'.format(day_shift)] = rmsle_by_logs(
            predicted_volume, test_sample[target_col_name])
    return metric_by_shift


drop_cols = [
    'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]
metric_by_shift = eval_model(
    dummy_train.drop(drop_cols, 1),
    'log_neg_amount', clf, [30])
for elem in metric_by_shift:
    print('{0}: {1}'.format(elem, metric_by_shift[elem]))

  0%|          | 0/1 [00:00<?, ?it/s]


CatboostError: Invalid param `eval_set`.

In [ ]:
dummy_train = pd.get_dummies(train_transactions, columns=['week_day', 'mcc_code'])
dummy_test = pd.get_dummies(test_transactions, columns=['week_day', 'mcc_code'])

# clf = XGBRegressor(learning_rate=0.01, max_depth=5, n_estimators=1300, nthread=-1, reg_alpha=1, reg_lambda=1)

drop_cols = [
    'month_num',
    'mcc_mean', 'mcc_min', 'mcc_max', 'mcc_std', 'mcc_median'
]

for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]
    
submit_table = get_agile_prediction(
    dummy_train, 
    dummy_train.log_neg_amount, 
    dummy_test.drop(drop_cols, 1), 
    clf, test_transactions)
submit_table.volume = submit_table.volume.apply(lambda x: 0 if x<0 else x)
submit_table.to_csv('agile_n_trans_xgb_neg-1300.csv', index=False)
submit_table.head()

In [12]:
# train_transactions['mcc_mean'] = train_transactions.mcc_code.map(
#     train_transactions.groupby('mcc_code').mean()['log_neg_amount'])
# test_transactions['mcc_mean'] = test_transactions.mcc_code.map(
#     train_transactions.groupby('mcc_code').mean()['log_neg_amount'])

# train_transactions['mcc_median'] = train_transactions.mcc_code.map(
#     train_transactions.groupby('mcc_code').median()['log_neg_amount'])
# test_transactions['mcc_median'] = test_transactions.mcc_code.map(
#     train_transactions.groupby('mcc_code').median()['log_neg_amount'])

# train_transactions['mcc_std'] = train_transactions.mcc_code.map(
#     train_transactions.groupby('mcc_code').std()['log_neg_amount'])
# test_transactions['mcc_std'] = test_transactions.mcc_code.map(
#     train_transactions.groupby('mcc_code').std()['log_neg_amount'])

# train_transactions['mcc_min'] = train_transactions.mcc_code.map(
#     train_transactions.groupby('mcc_code').min()['log_neg_amount'])
# test_transactions['mcc_min'] = test_transactions.mcc_code.map(
#     train_transactions.groupby('mcc_code').min()['log_neg_amount'])

# train_transactions['mcc_max'] = train_transactions.mcc_code.map(
#     train_transactions.groupby('mcc_code').max()['log_neg_amount'])
# test_transactions['mcc_max'] = test_transactions.mcc_code.map(
#     train_transactions.groupby('mcc_code').max()['log_neg_amount'])

dummy_train = pd.get_dummies(train_transactions, columns=['week_day', 'mcc_code'])
dummy_test = pd.get_dummies(test_transactions, columns=['week_day', 'mcc_code'])

from xgboost import XGBRegressor
clf = XGBRegressor(learning_rate=0.02, max_depth=6, n_estimators=350, nthread=-1, reg_alpha=1, reg_lambda=1)
drop_cols = [
    'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount', 'week_num', 'month_num',
    'mcc_mean', 'mcc_min', 'mcc_max', 'mcc_std', 'mcc_median'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]
metric_by_shift = eval_model(
    dummy_train.drop(drop_cols, 1),
    'log_neg_amount', clf, [30])
for elem in metric_by_shift:
    print('{0}: {1}'.format(elem, metric_by_shift[elem]))

100%|██████████| 1/1 [01:09<00:00, 69.52s/it]

RMSLE with 30 days: 3.303294930724346


In [18]:
dummy_train = pd.get_dummies(train_transactions, columns=['week_day', 'mcc_code'])
dummy_test = pd.get_dummies(test_transactions, columns=['week_day', 'mcc_code'])

clf = XGBRegressor(learning_rate=0.01, max_depth=5, n_estimators=1300, nthread=-1, reg_alpha=1, reg_lambda=1)

drop_cols = [
    'month_num',
    'mcc_mean', 'mcc_min', 'mcc_max', 'mcc_std', 'mcc_median'
]

for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]
    
submit_table = get_agile_prediction(
    dummy_train, 
    dummy_train.log_neg_amount, 
    dummy_test.drop(drop_cols, 1), 
    clf, test_transactions)
submit_table.volume = submit_table.volume.apply(lambda x: 0 if x<0 else x)
submit_table.to_csv('agile_n_trans_xgb_neg-1300.csv', index=False)
submit_table.head()

  0%|          | 0/5 [00:00<?, ?it/s]

[0]	validation_0-rmse:9.89464	validation_1-rmse:10.2383
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:9.80847	validation_1-rmse:10.1472
[2]	validation_0-rmse:9.72326	validation_1-rmse:10.0572
[3]	validation_0-rmse:9.63902	validation_1-rmse:9.96812
[4]	validation_0-rmse:9.55574	validation_1-rmse:9.88006
[5]	validation_0-rmse:9.4734	validation_1-rmse:9.79303
[6]	validation_0-rmse:9.39198	validation_1-rmse:9.70682
[7]	validation_0-rmse:9.3115	validation_1-rmse:9.62172
[8]	validation_0-rmse:9.23192	validation_1-rmse:9.5374
[9]	validation_0-rmse:9.15326	validation_1-rmse:9.45405
[10]	validation_0-rmse:9.0755	validation_1-rmse:9.3716
[11]	validation_0-rmse:8.99862	validation_1-rmse:9.29038
[12]	validation_0-rmse:8.92262	validation_1-rmse:9.20975
[13]	validation_0-rmse:8.84749	validation_1-rmse:9.13018
[14]	validation_0-rmse:8.77325	validation_1-rmse:9.05153


[141]	validation_0-rmse:4.14103	validation_1-rmse:4.07254
[142]	validation_0-rmse:4.12854	validation_1-rmse:4.05888
[143]	validation_0-rmse:4.11624	validation_1-rmse:4.04542
[144]	validation_0-rmse:4.10412	validation_1-rmse:4.0321
[145]	validation_0-rmse:4.09222	validation_1-rmse:4.01899
[146]	validation_0-rmse:4.08055	validation_1-rmse:4.00617
[147]	validation_0-rmse:4.06904	validation_1-rmse:3.99358
[148]	validation_0-rmse:4.0577	validation_1-rmse:3.98116
[149]	validation_0-rmse:4.04656	validation_1-rmse:3.96898
[150]	validation_0-rmse:4.03564	validation_1-rmse:3.95694
[151]	validation_0-rmse:4.02484	validation_1-rmse:3.94513
[152]	validation_0-rmse:4.01431	validation_1-rmse:3.93362
[153]	validation_0-rmse:4.00389	validation_1-rmse:3.92223
[154]	validation_0-rmse:3.99368	validation_1-rmse:3.911
[155]	validation_0-rmse:3.98359	validation_1-rmse:3.89993
[156]	validation_0-rmse:3.97365	validation_1-rmse:3.88907
[157]	validation_0-rmse:3.96394	validation_1-rmse:3.87856
[158]	validation_0

[283]	validation_0-rmse:3.4701	validation_1-rmse:3.35267
[284]	validation_0-rmse:3.469	validation_1-rmse:3.3517
[285]	validation_0-rmse:3.4679	validation_1-rmse:3.35066
[286]	validation_0-rmse:3.46687	validation_1-rmse:3.34971
[287]	validation_0-rmse:3.46579	validation_1-rmse:3.34879
[288]	validation_0-rmse:3.46478	validation_1-rmse:3.3479
[289]	validation_0-rmse:3.46372	validation_1-rmse:3.34697
[290]	validation_0-rmse:3.46271	validation_1-rmse:3.34605
[291]	validation_0-rmse:3.46171	validation_1-rmse:3.34515
[292]	validation_0-rmse:3.46073	validation_1-rmse:3.34426
[293]	validation_0-rmse:3.45974	validation_1-rmse:3.34337
[294]	validation_0-rmse:3.45883	validation_1-rmse:3.34244
[295]	validation_0-rmse:3.45787	validation_1-rmse:3.34167
[296]	validation_0-rmse:3.45693	validation_1-rmse:3.34094
[297]	validation_0-rmse:3.45599	validation_1-rmse:3.3401
[298]	validation_0-rmse:3.45509	validation_1-rmse:3.33927
[299]	validation_0-rmse:3.45421	validation_1-rmse:3.33857
[300]	validation_0-rm

[425]	validation_0-rmse:3.39503	validation_1-rmse:3.30186
[426]	validation_0-rmse:3.39473	validation_1-rmse:3.30186
[427]	validation_0-rmse:3.39447	validation_1-rmse:3.30186
[428]	validation_0-rmse:3.39415	validation_1-rmse:3.30173
[429]	validation_0-rmse:3.3939	validation_1-rmse:3.30159
[430]	validation_0-rmse:3.39364	validation_1-rmse:3.3014
[431]	validation_0-rmse:3.39341	validation_1-rmse:3.30131
[432]	validation_0-rmse:3.39311	validation_1-rmse:3.30133
[433]	validation_0-rmse:3.39281	validation_1-rmse:3.30125
[434]	validation_0-rmse:3.39256	validation_1-rmse:3.30126
[435]	validation_0-rmse:3.39228	validation_1-rmse:3.30114
[436]	validation_0-rmse:3.39202	validation_1-rmse:3.30105
[437]	validation_0-rmse:3.39173	validation_1-rmse:3.30112
[438]	validation_0-rmse:3.39148	validation_1-rmse:3.30113
[439]	validation_0-rmse:3.39124	validation_1-rmse:3.30095
[440]	validation_0-rmse:3.391	validation_1-rmse:3.30082
[441]	validation_0-rmse:3.39072	validation_1-rmse:3.30091
[442]	validation_0

...

[852]	validation_0-rmse:3.55072	validation_1-rmse:3.35424
[853]	validation_0-rmse:3.55057	validation_1-rmse:3.35421
[854]	validation_0-rmse:3.55039	validation_1-rmse:3.35425
[855]	validation_0-rmse:3.55022	validation_1-rmse:3.35425
[856]	validation_0-rmse:3.55005	validation_1-rmse:3.35426
[857]	validation_0-rmse:3.54985	validation_1-rmse:3.35436
[858]	validation_0-rmse:3.54973	validation_1-rmse:3.35441
[859]	validation_0-rmse:3.54954	validation_1-rmse:3.35431
[860]	validation_0-rmse:3.54943	validation_1-rmse:3.35427
[861]	validation_0-rmse:3.54933	validation_1-rmse:3.35424
[862]	validation_0-rmse:3.54917	validation_1-rmse:3.35418
[863]	validation_0-rmse:3.54901	validation_1-rmse:3.35421
[864]	validation_0-rmse:3.54889	validation_1-rmse:3.35421
[865]	validation_0-rmse:3.54875	validation_1-rmse:3.35417
[866]	validation_0-rmse:3.54858	validation_1-rmse:3.35412
[867]	validation_0-rmse:3.5484	validation_1-rmse:3.35417
[868]	validation_0-rmse:3.54824	validation_1-rmse:3.35413
[869]	validatio

[994]	validation_0-rmse:3.52994	validation_1-rmse:3.3534
[995]	validation_0-rmse:3.52978	validation_1-rmse:3.35341
[996]	validation_0-rmse:3.52965	validation_1-rmse:3.3534
[997]	validation_0-rmse:3.52957	validation_1-rmse:3.35332
[998]	validation_0-rmse:3.52944	validation_1-rmse:3.35331
[999]	validation_0-rmse:3.5293	validation_1-rmse:3.35333
[1000]	validation_0-rmse:3.52919	validation_1-rmse:3.35331
[1001]	validation_0-rmse:3.52904	validation_1-rmse:3.35333
[1002]	validation_0-rmse:3.52894	validation_1-rmse:3.35335
[1003]	validation_0-rmse:3.52882	validation_1-rmse:3.35327
[1004]	validation_0-rmse:3.52871	validation_1-rmse:3.35328
[1005]	validation_0-rmse:3.52858	validation_1-rmse:3.35324
[1006]	validation_0-rmse:3.52844	validation_1-rmse:3.35317
[1007]	validation_0-rmse:3.52829	validation_1-rmse:3.35318
[1008]	validation_0-rmse:3.52817	validation_1-rmse:3.35317
[1009]	validation_0-rmse:3.52806	validation_1-rmse:3.35317
[1010]	validation_0-rmse:3.52784	validation_1-rmse:3.3532
[1011]	

[1134]	validation_0-rmse:3.51159	validation_1-rmse:3.3533
[1135]	validation_0-rmse:3.51149	validation_1-rmse:3.35324
[1136]	validation_0-rmse:3.51132	validation_1-rmse:3.35351
[1137]	validation_0-rmse:3.51119	validation_1-rmse:3.35351
[1138]	validation_0-rmse:3.51107	validation_1-rmse:3.35347
[1139]	validation_0-rmse:3.51096	validation_1-rmse:3.35347
[1140]	validation_0-rmse:3.51084	validation_1-rmse:3.35344
[1141]	validation_0-rmse:3.51072	validation_1-rmse:3.35353
[1142]	validation_0-rmse:3.5106	validation_1-rmse:3.35356
[1143]	validation_0-rmse:3.5105	validation_1-rmse:3.35355
[1144]	validation_0-rmse:3.51034	validation_1-rmse:3.35343
[1145]	validation_0-rmse:3.51016	validation_1-rmse:3.35337
[1146]	validation_0-rmse:3.51005	validation_1-rmse:3.3533
[1147]	validation_0-rmse:3.50989	validation_1-rmse:3.35327
[1148]	validation_0-rmse:3.50978	validation_1-rmse:3.35325
[1149]	validation_0-rmse:3.50967	validation_1-rmse:3.35322
[1150]	validation_0-rmse:3.50953	validation_1-rmse:3.3532
[1

100%|██████████| 5/5 [24:32<00:00, 302.04s/it]


,id,volume
0,4814-457,7957318.0
1,4814-458,10201976.0
2,4814-459,14712970.0
3,4814-460,10261145.0
4,4814-461,10261145.0


In [19]:
def rmsle(predicted, actual):
    assert(len(predicted) == len(actual))
    p = np.log(np.array(predicted) + 1)
    a = np.log(np.array(actual) + 1)
    return (((p - a)**2).sum() / len(predicted))**0.5

def rmsle_by_logs(predicted, actual):
    assert(len(predicted) == len(actual))
    return (((predicted - actual)**2).sum() / len(predicted))**0.5

def eval_model(labeled_data, target_col_name, clf, day_shifts=np.arange(90, 0, -15)):
    max_day = labeled_data.day.max()
    c = labeled_data.columns.difference([target_col_name])
    metric_by_shift = {}
    for day_shift in tqdm(day_shifts): 
        train_sample = labeled_data[labeled_data.day <= max_day - day_shift]
        test_sample = labeled_data[labeled_data.day > max_day - day_shift]
        clf.fit(train_sample[c], train_sample[target_col_name])
        predicted_volume = clf.predict(test_sample[c])
        metric_by_shift['RMSLE with {} days'.format(day_shift)] = rmsle_by_logs(
            predicted_volume, test_sample[target_col_name])
    return metric_by_shift

def test_weeks_preparation(X_test):
    X_test_by_weeks = []
    current_test_week = X_test[X_test.week_num == 65].copy()
    X_test_by_weeks.append(current_test_week)
    removing_month_cols = ['prev_month_1_mean_neg', 'prev_month_1_std_neg', 'prev_month_1_median_neg', 
                           'prev_month_1_max_neg', 'prev_month_1_min_neg',
#                            'prev_month_1_mean_pos', 'prev_month_1_std_pos', 'prev_month_1_median_pos'
                          ]
    removing_week_cols = []
    for prev_week_index in range(1, 5):
        current_test_week = X_test[X_test.week_num == 65 + prev_week_index].copy()
        current_test_week.drop(removing_month_cols, 1, inplace=True)
        removing_week_cols += [
            'prev_week_{}_neg'.format(prev_week_index),
            'prev_week_{}_mean_neg'.format(prev_week_index),
            'prev_week_{}_std_neg'.format(prev_week_index),
            'prev_week_{}_n_trans'.format(prev_week_index),
            'prev_week_{}_mean_n_trans'.format(prev_week_index),
            'prev_week_{}_std_n_trans'.format(prev_week_index),
            
        ]
        current_test_week.drop(removing_week_cols, 1, inplace=True)
        X_test_by_weeks.append(current_test_week)
    return X_test_by_weeks
        
def get_agile_prediction(X_train, y_train, X_test, clf, test_form):
    X_test_by_weeks = test_weeks_preparation(X_test)
    out_form = pd.DataFrame()
    for prev_week_index in tqdm(range(5)):
        current_test = X_test_by_weeks[prev_week_index] 
        drop_cols = X_train.columns.difference(current_test.columns)
        current_train = X_train.drop(drop_cols, 1)
        clf.fit(current_train, y_train)
        predicted_log_volume = clf.predict(current_test)
        current_out_form = test_form[test_form.week_num == 65 + prev_week_index].copy()
        current_out_form['id'] = current_out_form[['mcc_code', 'day']].apply(
            lambda x: '-'.join(map(str, x)), axis=1)
        current_out_form['volume'] = np.expm1(predicted_log_volume)
        out_form = out_form.append(current_out_form[['id', 'volume']])
    return out_form

In [20]:
dummy_train = pd.get_dummies(train_transactions, columns=['week_day', 'mcc_code'])
dummy_test = pd.get_dummies(test_transactions, columns=['week_day', 'mcc_code'])

clf = XGBRegressor(learning_rate=0.02, max_depth=6, n_estimators=350, nthread=-1, reg_alpha=1, reg_lambda=1)

drop_cols = [
    'month_num',
    'mcc_mean', 'mcc_min', 'mcc_max', 'mcc_std', 'mcc_median'
]

for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]
    
submit_table = get_agile_prediction(
    dummy_train, 
    dummy_train.log_neg_amount, 
    dummy_test.drop(drop_cols, 1), 
    clf, test_transactions)
submit_table.volume = submit_table.volume.apply(lambda x: 0 if x<0 else x)
submit_table.to_csv('agile_n_trans_xgb_neg-dummyweek-350.csv', index=False)
submit_table.head()

100%|██████████| 5/5 [05:31<00:00, 66.03s/it]


,id,volume
0,4814-457,8340080.5
1,4814-458,10421065.0
2,4814-459,14945321.0
3,4814-460,10511917.0
4,4814-461,10511917.0


In [9]:
from xgboost import XGBRegressor
clf = XGBRegressor(learning_rate=0.02, max_depth=6, n_estimators=350, nthread=-1, reg_alpha=1, reg_lambda=1)
drop_cols = [
    'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]
    
drop_cols += [col for col in dummy_train.columns if 'mcc_code_' in col]
drop_cols += []

metric_by_shift = eval_model(
    dummy_train.drop(drop_cols, 1),
    'log_neg_amount', clf, [30])
for elem in metric_by_shift:
    print('{0}: {1}'.format(elem, metric_by_shift[elem]))

100%|██████████| 1/1 [00:45<00:00, 45.12s/it]

RMSLE with 30 days: 3.307794835891832


In [11]:
from xgboost import XGBRegressor
clf = XGBRegressor(learning_rate=0.02, max_depth=6, n_estimators=1350, nthread=-1, reg_alpha=1, reg_lambda=1)
drop_cols = [
    'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]
# drop_cols += [col for col in dummy_train.columns if 'mcc_code_' in col]
labeled_data, target_col_name = dummy_train.drop(drop_cols, 1), 'log_neg_amount'
max_day = labeled_data.day.max()
c = labeled_data.columns.difference([target_col_name])
metric_by_shift = {}
day_shift = 30 
train_sample = labeled_data[labeled_data.day <= max_day - day_shift]
test_sample = labeled_data[labeled_data.day > max_day - day_shift]

eval_set  = [(test_sample[c],test_sample[target_col_name])]
clf.fit(train_sample[c], train_sample[target_col_name], 
        early_stopping_rounds=50, eval_metric='rmse',
        eval_set=eval_set)

predicted_volume = clf.predict(test_sample[c])
metric_by_shift['RMSLE with {} days'.format(day_shift)] = rmsle_by_logs(
    predicted_volume, test_sample[target_col_name])
print(metric_by_shift)

[0]	validation_0-rmse:10.1461
Will train until validation_0-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:9.96577
[2]	validation_0-rmse:9.78973
[3]	validation_0-rmse:9.61724
[4]	validation_0-rmse:9.44844
[5]	validation_0-rmse:9.28391
[6]	validation_0-rmse:9.12268
[7]	validation_0-rmse:8.96554
[8]	validation_0-rmse:8.81191
[9]	validation_0-rmse:8.66159
[10]	validation_0-rmse:8.51465
[11]	validation_0-rmse:8.37113
[12]	validation_0-rmse:8.23096
[13]	validation_0-rmse:8.09364
[14]	validation_0-rmse:7.95994
[15]	validation_0-rmse:7.8294
[16]	validation_0-rmse:7.70194
[17]	validation_0-rmse:7.57724
[18]	validation_0-rmse:7.4557
[19]	validation_0-rmse:7.33698
[20]	validation_0-rmse:7.22139
[21]	validation_0-rmse:7.10784
[22]	validation_0-rmse:6.99757
[23]	validation_0-rmse:6.8901
[24]	validation_0-rmse:6.78538
[25]	validation_0-rmse:6.68238
[26]	validation_0-rmse:6.58261
[27]	validation_0-rmse:6.48513
[28]	validation_0-rmse:6.39029
[29]	validation_0-rmse:6.29751
[30]	validation_0-

[259]	validation_0-rmse:3.30354
[260]	validation_0-rmse:3.30341
[261]	validation_0-rmse:3.30344
[262]	validation_0-rmse:3.3037
[263]	validation_0-rmse:3.30379
[264]	validation_0-rmse:3.30378
[265]	validation_0-rmse:3.30385
[266]	validation_0-rmse:3.30406
[267]	validation_0-rmse:3.30417
[268]	validation_0-rmse:3.30418
[269]	validation_0-rmse:3.30421
[270]	validation_0-rmse:3.3043
[271]	validation_0-rmse:3.30434
[272]	validation_0-rmse:3.30444
[273]	validation_0-rmse:3.30438
[274]	validation_0-rmse:3.30438
[275]	validation_0-rmse:3.30438
[276]	validation_0-rmse:3.30457
[277]	validation_0-rmse:3.30455
[278]	validation_0-rmse:3.30454
[279]	validation_0-rmse:3.30459
[280]	validation_0-rmse:3.30462
[281]	validation_0-rmse:3.30467
[282]	validation_0-rmse:3.30478
[283]	validation_0-rmse:3.30486
[284]	validation_0-rmse:3.30476
[285]	validation_0-rmse:3.30467
[286]	validation_0-rmse:3.30464
[287]	validation_0-rmse:3.30443
[288]	validation_0-rmse:3.30438
[289]	validation_0-rmse:3.30424
[290]	vali

In [47]:
from xgboost import XGBRegressor
clf = XGBRegressor(learning_rate=0.02, max_depth=6, n_estimators=1350, nthread=-1, reg_alpha=1, reg_lambda=1)
drop_cols = [
    'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]

labeled_data, target_col_name = dummy_train.drop(drop_cols, 1), 'log_neg_amount'
max_day = labeled_data.day.max()
c = labeled_data.columns.difference([target_col_name])
metric_by_shift = {}
day_shift = 30 
train_sample = labeled_data[labeled_data.day <= max_day - day_shift]
test_sample = labeled_data[labeled_data.day > max_day - day_shift]

eval_set  = [(test_sample[c],test_sample[target_col_name])]
clf.fit(train_sample[c], train_sample[target_col_name], 
        early_stopping_rounds=50, eval_metric='rmse',
        eval_set=eval_set)

predicted_volume = clf.predict(test_sample[c])
metric_by_shift['RMSLE with {} days'.format(day_shift)] = rmsle_by_logs(
    predicted_volume, test_sample[target_col_name])
print(metric_by_shift)

[0]	validation_0-rmse:10.1461
Will train until validation_0-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:9.96577
[2]	validation_0-rmse:9.78973
[3]	validation_0-rmse:9.61724
[4]	validation_0-rmse:9.44844
[5]	validation_0-rmse:9.28391
[6]	validation_0-rmse:9.12268
[7]	validation_0-rmse:8.96554
[8]	validation_0-rmse:8.81191
[9]	validation_0-rmse:8.66159
[10]	validation_0-rmse:8.51465
[11]	validation_0-rmse:8.37113
[12]	validation_0-rmse:8.23096
[13]	validation_0-rmse:8.09364
[14]	validation_0-rmse:7.95994
[15]	validation_0-rmse:7.8294
[16]	validation_0-rmse:7.70194
[17]	validation_0-rmse:7.57724
[18]	validation_0-rmse:7.4557
[19]	validation_0-rmse:7.33698
[20]	validation_0-rmse:7.22139
[21]	validation_0-rmse:7.10784
[22]	validation_0-rmse:6.99757
[23]	validation_0-rmse:6.8901
[24]	validation_0-rmse:6.78538
[25]	validation_0-rmse:6.68238
[26]	validation_0-rmse:6.58261
[27]	validation_0-rmse:6.48513
[28]	validation_0-rmse:6.39029
[29]	validation_0-rmse:6.29751
[30]	validation_0-

[259]	validation_0-rmse:3.30354
[260]	validation_0-rmse:3.30341
[261]	validation_0-rmse:3.30344
[262]	validation_0-rmse:3.3037
[263]	validation_0-rmse:3.30379
[264]	validation_0-rmse:3.30378
[265]	validation_0-rmse:3.30385
[266]	validation_0-rmse:3.30406
[267]	validation_0-rmse:3.30417
[268]	validation_0-rmse:3.30418
[269]	validation_0-rmse:3.30421
[270]	validation_0-rmse:3.3043
[271]	validation_0-rmse:3.30434
[272]	validation_0-rmse:3.30444
[273]	validation_0-rmse:3.30438
[274]	validation_0-rmse:3.30438
[275]	validation_0-rmse:3.30438
[276]	validation_0-rmse:3.30457
[277]	validation_0-rmse:3.30455
[278]	validation_0-rmse:3.30454
[279]	validation_0-rmse:3.30459
[280]	validation_0-rmse:3.30462
[281]	validation_0-rmse:3.30467
[282]	validation_0-rmse:3.30478
[283]	validation_0-rmse:3.30486
[284]	validation_0-rmse:3.30476
[285]	validation_0-rmse:3.30467
[286]	validation_0-rmse:3.30464
[287]	validation_0-rmse:3.30443
[288]	validation_0-rmse:3.30438
[289]	validation_0-rmse:3.30424
[290]	vali

In [28]:
from xgboost import XGBRegressor
clf = XGBRegressor(learning_rate=0.01, max_depth=5, n_estimators=3000, nthread=-1, reg_alpha=1, reg_lambda=1)
drop_cols = [
    'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]

labeled_data, target_col_name = dummy_train.drop(drop_cols, 1), 'log_neg_amount'
max_day = labeled_data.day.max()
c = labeled_data.columns.difference([target_col_name])
metric_by_shift = {}
day_shift = 30 
train_sample = labeled_data[labeled_data.day <= max_day - day_shift]
test_sample = labeled_data[labeled_data.day > max_day - day_shift]

eval_set  = [(train_sample[c], train_sample[target_col_name]), (test_sample[c],test_sample[target_col_name])]
clf.fit(train_sample[c], train_sample[target_col_name], 
        early_stopping_rounds=50, eval_metric='rmse',
        eval_set=eval_set)

predicted_volume = clf.predict(test_sample[c])
metric_by_shift['RMSLE with {} days'.format(day_shift)] = rmsle_by_logs(
    predicted_volume, test_sample[target_col_name])
print(metric_by_shift)

[0]	validation_0-rmse:9.89464	validation_1-rmse:10.2383
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:9.80847	validation_1-rmse:10.1472
[2]	validation_0-rmse:9.72326	validation_1-rmse:10.0572
[3]	validation_0-rmse:9.63902	validation_1-rmse:9.96812
[4]	validation_0-rmse:9.55574	validation_1-rmse:9.88006
[5]	validation_0-rmse:9.4734	validation_1-rmse:9.79303
[6]	validation_0-rmse:9.39198	validation_1-rmse:9.70682
[7]	validation_0-rmse:9.3115	validation_1-rmse:9.62172
[8]	validation_0-rmse:9.23192	validation_1-rmse:9.5374
[9]	validation_0-rmse:9.15326	validation_1-rmse:9.45405
[10]	validation_0-rmse:9.0755	validation_1-rmse:9.3716
[11]	validation_0-rmse:8.99862	validation_1-rmse:9.29038
[12]	validation_0-rmse:8.92262	validation_1-rmse:9.20975
[13]	validation_0-rmse:8.84749	validation_1-rmse:9.13018
[14]	validation_0-rmse:8.77325	validation_1-rmse:9.05153
[

[142]	validation_0-rmse:4.12855	validation_1-rmse:4.05886
[143]	validation_0-rmse:4.11625	validation_1-rmse:4.04539
[144]	validation_0-rmse:4.10414	validation_1-rmse:4.0321
[145]	validation_0-rmse:4.0923	validation_1-rmse:4.01915
[146]	validation_0-rmse:4.08054	validation_1-rmse:4.00632
[147]	validation_0-rmse:4.06902	validation_1-rmse:3.99365
[148]	validation_0-rmse:4.05768	validation_1-rmse:3.98125
[149]	validation_0-rmse:4.04656	validation_1-rmse:3.96893
[150]	validation_0-rmse:4.03564	validation_1-rmse:3.9569
[151]	validation_0-rmse:4.02491	validation_1-rmse:3.94513
[152]	validation_0-rmse:4.01431	validation_1-rmse:3.93354
[153]	validation_0-rmse:4.00392	validation_1-rmse:3.92212
[154]	validation_0-rmse:3.99371	validation_1-rmse:3.911
[155]	validation_0-rmse:3.98362	validation_1-rmse:3.90002
[156]	validation_0-rmse:3.97371	validation_1-rmse:3.88918
[157]	validation_0-rmse:3.964	validation_1-rmse:3.87864
[158]	validation_0-rmse:3.95441	validation_1-rmse:3.86804
[159]	validation_0-rm

[284]	validation_0-rmse:3.46915	validation_1-rmse:3.35115
[285]	validation_0-rmse:3.46807	validation_1-rmse:3.35022
[286]	validation_0-rmse:3.467	validation_1-rmse:3.34922
[287]	validation_0-rmse:3.46593	validation_1-rmse:3.34817
[288]	validation_0-rmse:3.46486	validation_1-rmse:3.34719
[289]	validation_0-rmse:3.46388	validation_1-rmse:3.34628
[290]	validation_0-rmse:3.46287	validation_1-rmse:3.34536
[291]	validation_0-rmse:3.4619	validation_1-rmse:3.34439
[292]	validation_0-rmse:3.4609	validation_1-rmse:3.34347
[293]	validation_0-rmse:3.45994	validation_1-rmse:3.34271
[294]	validation_0-rmse:3.45898	validation_1-rmse:3.34186
[295]	validation_0-rmse:3.45804	validation_1-rmse:3.34101
[296]	validation_0-rmse:3.45709	validation_1-rmse:3.34021
[297]	validation_0-rmse:3.4562	validation_1-rmse:3.33936
[298]	validation_0-rmse:3.45525	validation_1-rmse:3.33854
[299]	validation_0-rmse:3.45439	validation_1-rmse:3.33778
[300]	validation_0-rmse:3.4535	validation_1-rmse:3.33702
[301]	validation_0-r

[426]	validation_0-rmse:3.39496	validation_1-rmse:3.30224
[427]	validation_0-rmse:3.39469	validation_1-rmse:3.3022
[428]	validation_0-rmse:3.39447	validation_1-rmse:3.30209
[429]	validation_0-rmse:3.39416	validation_1-rmse:3.30199
[430]	validation_0-rmse:3.39386	validation_1-rmse:3.302
[431]	validation_0-rmse:3.3936	validation_1-rmse:3.30196
[432]	validation_0-rmse:3.39333	validation_1-rmse:3.30186
[433]	validation_0-rmse:3.39304	validation_1-rmse:3.30185
[434]	validation_0-rmse:3.39276	validation_1-rmse:3.30182
[435]	validation_0-rmse:3.39247	validation_1-rmse:3.30186
[436]	validation_0-rmse:3.39222	validation_1-rmse:3.30181
[437]	validation_0-rmse:3.39195	validation_1-rmse:3.30173
[438]	validation_0-rmse:3.39167	validation_1-rmse:3.30155
[439]	validation_0-rmse:3.39142	validation_1-rmse:3.30155
[440]	validation_0-rmse:3.39113	validation_1-rmse:3.30149
[441]	validation_0-rmse:3.39085	validation_1-rmse:3.30159
[442]	validation_0-rmse:3.39061	validation_1-rmse:3.30158
[443]	validation_0

[568]	validation_0-rmse:3.36461	validation_1-rmse:3.29848
[569]	validation_0-rmse:3.36445	validation_1-rmse:3.29835
[570]	validation_0-rmse:3.36431	validation_1-rmse:3.29832
[571]	validation_0-rmse:3.36421	validation_1-rmse:3.29828
[572]	validation_0-rmse:3.36415	validation_1-rmse:3.29827
[573]	validation_0-rmse:3.36401	validation_1-rmse:3.29823
[574]	validation_0-rmse:3.36385	validation_1-rmse:3.29828
[575]	validation_0-rmse:3.36371	validation_1-rmse:3.29831
[576]	validation_0-rmse:3.3636	validation_1-rmse:3.29825
[577]	validation_0-rmse:3.36345	validation_1-rmse:3.29828
[578]	validation_0-rmse:3.36334	validation_1-rmse:3.29829
[579]	validation_0-rmse:3.36318	validation_1-rmse:3.29834
[580]	validation_0-rmse:3.36311	validation_1-rmse:3.29834
[581]	validation_0-rmse:3.36301	validation_1-rmse:3.29835
[582]	validation_0-rmse:3.36292	validation_1-rmse:3.29835
[583]	validation_0-rmse:3.36275	validation_1-rmse:3.29835
[584]	validation_0-rmse:3.36261	validation_1-rmse:3.29836
[585]	validatio

In [29]:
from xgboost import XGBRegressor
clf = XGBRegressor(learning_rate=0.01, max_depth=6, n_estimators=3000, nthread=-1, reg_alpha=1, reg_lambda=1)
drop_cols = [
    'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]

labeled_data, target_col_name = dummy_train.drop(drop_cols, 1), 'log_neg_amount'
max_day = labeled_data.day.max()
c = labeled_data.columns.difference([target_col_name])
metric_by_shift = {}
day_shift = 30 
train_sample = labeled_data[labeled_data.day <= max_day - day_shift]
test_sample = labeled_data[labeled_data.day > max_day - day_shift]

eval_set  = [(train_sample[c], train_sample[target_col_name]), (test_sample[c],test_sample[target_col_name])]
clf.fit(train_sample[c], train_sample[target_col_name], 
        early_stopping_rounds=50, eval_metric='rmse',
        eval_set=eval_set)

predicted_volume = clf.predict(test_sample[c])
metric_by_shift['RMSLE with {} days'.format(day_shift)] = rmsle_by_logs(
    predicted_volume, test_sample[target_col_name])
print(metric_by_shift)

[0]	validation_0-rmse:9.89444	validation_1-rmse:10.2382
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:9.80806	validation_1-rmse:10.147
[2]	validation_0-rmse:9.72266	validation_1-rmse:10.0568
[3]	validation_0-rmse:9.63822	validation_1-rmse:9.96763
[4]	validation_0-rmse:9.55473	validation_1-rmse:9.87945
[5]	validation_0-rmse:9.47219	validation_1-rmse:9.79228
[6]	validation_0-rmse:9.39058	validation_1-rmse:9.70603
[7]	validation_0-rmse:9.30988	validation_1-rmse:9.62053
[8]	validation_0-rmse:9.23012	validation_1-rmse:9.53626
[9]	validation_0-rmse:9.15123	validation_1-rmse:9.45293
[10]	validation_0-rmse:9.07326	validation_1-rmse:9.37046
[11]	validation_0-rmse:8.99618	validation_1-rmse:9.28878
[12]	validation_0-rmse:8.91997	validation_1-rmse:9.20825
[13]	validation_0-rmse:8.84462	validation_1-rmse:9.12858
[14]	validation_0-rmse:8.77016	validation_1-rmse:9.049

[141]	validation_0-rmse:4.11066	validation_1-rmse:4.07
[142]	validation_0-rmse:4.09796	validation_1-rmse:4.05633
[143]	validation_0-rmse:4.08552	validation_1-rmse:4.04286
[144]	validation_0-rmse:4.07319	validation_1-rmse:4.02962
[145]	validation_0-rmse:4.06111	validation_1-rmse:4.01655
[146]	validation_0-rmse:4.04925	validation_1-rmse:4.00378
[147]	validation_0-rmse:4.03753	validation_1-rmse:3.99123
[148]	validation_0-rmse:4.02597	validation_1-rmse:3.97885
[149]	validation_0-rmse:4.01462	validation_1-rmse:3.96656
[150]	validation_0-rmse:4.00351	validation_1-rmse:3.95457
[151]	validation_0-rmse:3.9926	validation_1-rmse:3.94289
[152]	validation_0-rmse:3.98178	validation_1-rmse:3.93125
[153]	validation_0-rmse:3.97119	validation_1-rmse:3.91995
[154]	validation_0-rmse:3.96081	validation_1-rmse:3.9088
[155]	validation_0-rmse:3.95053	validation_1-rmse:3.89773
[156]	validation_0-rmse:3.94047	validation_1-rmse:3.88697
[157]	validation_0-rmse:3.93052	validation_1-rmse:3.87639
[158]	validation_0-

[283]	validation_0-rmse:3.41473	validation_1-rmse:3.35273
[284]	validation_0-rmse:3.4135	validation_1-rmse:3.35175
[285]	validation_0-rmse:3.41227	validation_1-rmse:3.35075
[286]	validation_0-rmse:3.41107	validation_1-rmse:3.34965
[287]	validation_0-rmse:3.40989	validation_1-rmse:3.34877
[288]	validation_0-rmse:3.4087	validation_1-rmse:3.34788
[289]	validation_0-rmse:3.40757	validation_1-rmse:3.34698
[290]	validation_0-rmse:3.40642	validation_1-rmse:3.34614
[291]	validation_0-rmse:3.40531	validation_1-rmse:3.34536
[292]	validation_0-rmse:3.40421	validation_1-rmse:3.34442
[293]	validation_0-rmse:3.40317	validation_1-rmse:3.34354
[294]	validation_0-rmse:3.40208	validation_1-rmse:3.34271
[295]	validation_0-rmse:3.40105	validation_1-rmse:3.34198
[296]	validation_0-rmse:3.3999	validation_1-rmse:3.34116
[297]	validation_0-rmse:3.39888	validation_1-rmse:3.34027
[298]	validation_0-rmse:3.39788	validation_1-rmse:3.33955
[299]	validation_0-rmse:3.39683	validation_1-rmse:3.33879
[300]	validation_

[425]	validation_0-rmse:3.32626	validation_1-rmse:3.30358
[426]	validation_0-rmse:3.32595	validation_1-rmse:3.30362
[427]	validation_0-rmse:3.3256	validation_1-rmse:3.30358
[428]	validation_0-rmse:3.32522	validation_1-rmse:3.30351
[429]	validation_0-rmse:3.32496	validation_1-rmse:3.30337
[430]	validation_0-rmse:3.32459	validation_1-rmse:3.30332
[431]	validation_0-rmse:3.32426	validation_1-rmse:3.30327
[432]	validation_0-rmse:3.32394	validation_1-rmse:3.30318
[433]	validation_0-rmse:3.3236	validation_1-rmse:3.3032
[434]	validation_0-rmse:3.32324	validation_1-rmse:3.30302
[435]	validation_0-rmse:3.32293	validation_1-rmse:3.30297
[436]	validation_0-rmse:3.32254	validation_1-rmse:3.30295
[437]	validation_0-rmse:3.32222	validation_1-rmse:3.30293
[438]	validation_0-rmse:3.32185	validation_1-rmse:3.30283
[439]	validation_0-rmse:3.32147	validation_1-rmse:3.30279
[440]	validation_0-rmse:3.32114	validation_1-rmse:3.30284
[441]	validation_0-rmse:3.32079	validation_1-rmse:3.30276
[442]	validation_

In [30]:
from xgboost import XGBRegressor
clf = XGBRegressor(learning_rate=0.01, max_depth=4, n_estimators=3000, nthread=-1, reg_alpha=1, reg_lambda=1)
drop_cols = [
    'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]

labeled_data, target_col_name = dummy_train.drop(drop_cols, 1), 'log_neg_amount'
max_day = labeled_data.day.max()
c = labeled_data.columns.difference([target_col_name])
metric_by_shift = {}
day_shift = 30 
train_sample = labeled_data[labeled_data.day <= max_day - day_shift]
test_sample = labeled_data[labeled_data.day > max_day - day_shift]

eval_set  = [(train_sample[c], train_sample[target_col_name]), (test_sample[c],test_sample[target_col_name])]
clf.fit(train_sample[c], train_sample[target_col_name], 
        early_stopping_rounds=50, eval_metric='rmse',
        eval_set=eval_set)

predicted_volume = clf.predict(test_sample[c])
metric_by_shift['RMSLE with {} days'.format(day_shift)] = rmsle_by_logs(
    predicted_volume, test_sample[target_col_name])
print(metric_by_shift)

[0]	validation_0-rmse:9.89491	validation_1-rmse:10.2385
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:9.80901	validation_1-rmse:10.1475
[2]	validation_0-rmse:9.72407	validation_1-rmse:10.0575
[3]	validation_0-rmse:9.6401	validation_1-rmse:9.96856
[4]	validation_0-rmse:9.55709	validation_1-rmse:9.88073
[5]	validation_0-rmse:9.47499	validation_1-rmse:9.79386
[6]	validation_0-rmse:9.39386	validation_1-rmse:9.70791
[7]	validation_0-rmse:9.31363	validation_1-rmse:9.62297
[8]	validation_0-rmse:9.2343	validation_1-rmse:9.53897
[9]	validation_0-rmse:9.1559	validation_1-rmse:9.45617
[10]	validation_0-rmse:9.0784	validation_1-rmse:9.37401
[11]	validation_0-rmse:9.00177	validation_1-rmse:9.29281
[12]	validation_0-rmse:8.92604	validation_1-rmse:9.21254
[13]	validation_0-rmse:8.85118	validation_1-rmse:9.13339
[14]	validation_0-rmse:8.77717	validation_1-rmse:9.05478


[142]	validation_0-rmse:4.15559	validation_1-rmse:4.06677
[143]	validation_0-rmse:4.14336	validation_1-rmse:4.05312
[144]	validation_0-rmse:4.13134	validation_1-rmse:4.03978
[145]	validation_0-rmse:4.11956	validation_1-rmse:4.0267
[146]	validation_0-rmse:4.10796	validation_1-rmse:4.01381
[147]	validation_0-rmse:4.09653	validation_1-rmse:4.00125
[148]	validation_0-rmse:4.08531	validation_1-rmse:3.98878
[149]	validation_0-rmse:4.07426	validation_1-rmse:3.97647
[150]	validation_0-rmse:4.06341	validation_1-rmse:3.96447
[151]	validation_0-rmse:4.05275	validation_1-rmse:3.95263
[152]	validation_0-rmse:4.04226	validation_1-rmse:3.94096
[153]	validation_0-rmse:4.03195	validation_1-rmse:3.92951
[154]	validation_0-rmse:4.02179	validation_1-rmse:3.91831
[155]	validation_0-rmse:4.01182	validation_1-rmse:3.90726
[156]	validation_0-rmse:4.00201	validation_1-rmse:3.89635
[157]	validation_0-rmse:3.99238	validation_1-rmse:3.88558
[158]	validation_0-rmse:3.98293	validation_1-rmse:3.87509
[159]	validatio

[284]	validation_0-rmse:3.50746	validation_1-rmse:3.35802
[285]	validation_0-rmse:3.50642	validation_1-rmse:3.35695
[286]	validation_0-rmse:3.5054	validation_1-rmse:3.35595
[287]	validation_0-rmse:3.50439	validation_1-rmse:3.35495
[288]	validation_0-rmse:3.5034	validation_1-rmse:3.35405
[289]	validation_0-rmse:3.50244	validation_1-rmse:3.3531
[290]	validation_0-rmse:3.50148	validation_1-rmse:3.35221
[291]	validation_0-rmse:3.50053	validation_1-rmse:3.35132
[292]	validation_0-rmse:3.49961	validation_1-rmse:3.35032
[293]	validation_0-rmse:3.4987	validation_1-rmse:3.34952
[294]	validation_0-rmse:3.4978	validation_1-rmse:3.34864
[295]	validation_0-rmse:3.49692	validation_1-rmse:3.34774
[296]	validation_0-rmse:3.49606	validation_1-rmse:3.34686
[297]	validation_0-rmse:3.49521	validation_1-rmse:3.34605
[298]	validation_0-rmse:3.49437	validation_1-rmse:3.34531
[299]	validation_0-rmse:3.49355	validation_1-rmse:3.34438
[300]	validation_0-rmse:3.49274	validation_1-rmse:3.34369
[301]	validation_0-

[426]	validation_0-rmse:3.44249	validation_1-rmse:3.30674
[427]	validation_0-rmse:3.44229	validation_1-rmse:3.30664
[428]	validation_0-rmse:3.44207	validation_1-rmse:3.30653
[429]	validation_0-rmse:3.44186	validation_1-rmse:3.30642
[430]	validation_0-rmse:3.44166	validation_1-rmse:3.3063
[431]	validation_0-rmse:3.44147	validation_1-rmse:3.3062
[432]	validation_0-rmse:3.44129	validation_1-rmse:3.30605
[433]	validation_0-rmse:3.44109	validation_1-rmse:3.30594
[434]	validation_0-rmse:3.44087	validation_1-rmse:3.30588
[435]	validation_0-rmse:3.44067	validation_1-rmse:3.30578
[436]	validation_0-rmse:3.44048	validation_1-rmse:3.30571
[437]	validation_0-rmse:3.4403	validation_1-rmse:3.30568
[438]	validation_0-rmse:3.44009	validation_1-rmse:3.30561
[439]	validation_0-rmse:3.43991	validation_1-rmse:3.30551
[440]	validation_0-rmse:3.43971	validation_1-rmse:3.30546
[441]	validation_0-rmse:3.43951	validation_1-rmse:3.30537
[442]	validation_0-rmse:3.43934	validation_1-rmse:3.30527
[443]	validation_

[568]	validation_0-rmse:3.4219	validation_1-rmse:3.30036
[569]	validation_0-rmse:3.42183	validation_1-rmse:3.30036
[570]	validation_0-rmse:3.42166	validation_1-rmse:3.30034
[571]	validation_0-rmse:3.42155	validation_1-rmse:3.30029
[572]	validation_0-rmse:3.42149	validation_1-rmse:3.30029
[573]	validation_0-rmse:3.42141	validation_1-rmse:3.3003
[574]	validation_0-rmse:3.42127	validation_1-rmse:3.30031
[575]	validation_0-rmse:3.4212	validation_1-rmse:3.30034
[576]	validation_0-rmse:3.4211	validation_1-rmse:3.30028
[577]	validation_0-rmse:3.42103	validation_1-rmse:3.30029
[578]	validation_0-rmse:3.42092	validation_1-rmse:3.30023
[579]	validation_0-rmse:3.42086	validation_1-rmse:3.30023
[580]	validation_0-rmse:3.42075	validation_1-rmse:3.30018
[581]	validation_0-rmse:3.42066	validation_1-rmse:3.30017
[582]	validation_0-rmse:3.42059	validation_1-rmse:3.30011
[583]	validation_0-rmse:3.42051	validation_1-rmse:3.30009
[584]	validation_0-rmse:3.42045	validation_1-rmse:3.30012
[585]	validation_0

[710]	validation_0-rmse:3.41054	validation_1-rmse:3.29863
[711]	validation_0-rmse:3.41048	validation_1-rmse:3.2986
[712]	validation_0-rmse:3.4104	validation_1-rmse:3.29861
[713]	validation_0-rmse:3.41032	validation_1-rmse:3.29862
[714]	validation_0-rmse:3.41026	validation_1-rmse:3.29861
[715]	validation_0-rmse:3.4102	validation_1-rmse:3.29861
[716]	validation_0-rmse:3.41013	validation_1-rmse:3.29861
[717]	validation_0-rmse:3.41005	validation_1-rmse:3.2986
[718]	validation_0-rmse:3.40996	validation_1-rmse:3.29865
[719]	validation_0-rmse:3.40988	validation_1-rmse:3.29866
[720]	validation_0-rmse:3.40981	validation_1-rmse:3.29865
[721]	validation_0-rmse:3.40967	validation_1-rmse:3.29861
[722]	validation_0-rmse:3.4096	validation_1-rmse:3.29856
[723]	validation_0-rmse:3.40952	validation_1-rmse:3.29855
[724]	validation_0-rmse:3.40943	validation_1-rmse:3.29854
[725]	validation_0-rmse:3.40937	validation_1-rmse:3.29852
[726]	validation_0-rmse:3.40928	validation_1-rmse:3.29858
[727]	validation_0-

[852]	validation_0-rmse:3.40035	validation_1-rmse:3.29808
[853]	validation_0-rmse:3.4003	validation_1-rmse:3.29808
[854]	validation_0-rmse:3.40025	validation_1-rmse:3.29811
[855]	validation_0-rmse:3.40019	validation_1-rmse:3.29808
[856]	validation_0-rmse:3.40013	validation_1-rmse:3.29803
[857]	validation_0-rmse:3.40008	validation_1-rmse:3.29804
[858]	validation_0-rmse:3.39998	validation_1-rmse:3.29803
[859]	validation_0-rmse:3.39991	validation_1-rmse:3.29811
[860]	validation_0-rmse:3.39987	validation_1-rmse:3.29807
[861]	validation_0-rmse:3.39977	validation_1-rmse:3.29808
[862]	validation_0-rmse:3.39972	validation_1-rmse:3.29807
[863]	validation_0-rmse:3.39966	validation_1-rmse:3.29811
[864]	validation_0-rmse:3.39961	validation_1-rmse:3.29811
[865]	validation_0-rmse:3.39956	validation_1-rmse:3.29809
[866]	validation_0-rmse:3.39951	validation_1-rmse:3.29807
[867]	validation_0-rmse:3.39943	validation_1-rmse:3.29806
[868]	validation_0-rmse:3.39938	validation_1-rmse:3.29809
[869]	validatio

[994]	validation_0-rmse:3.39124	validation_1-rmse:3.2971
[995]	validation_0-rmse:3.39118	validation_1-rmse:3.29711
[996]	validation_0-rmse:3.39111	validation_1-rmse:3.29711
[997]	validation_0-rmse:3.39105	validation_1-rmse:3.29713
[998]	validation_0-rmse:3.391	validation_1-rmse:3.29718
[999]	validation_0-rmse:3.39096	validation_1-rmse:3.29717
[1000]	validation_0-rmse:3.39091	validation_1-rmse:3.29716
[1001]	validation_0-rmse:3.39076	validation_1-rmse:3.29715
[1002]	validation_0-rmse:3.39066	validation_1-rmse:3.29711
[1003]	validation_0-rmse:3.39061	validation_1-rmse:3.2971
[1004]	validation_0-rmse:3.39047	validation_1-rmse:3.29706
[1005]	validation_0-rmse:3.39041	validation_1-rmse:3.29706
[1006]	validation_0-rmse:3.39036	validation_1-rmse:3.29704
[1007]	validation_0-rmse:3.3903	validation_1-rmse:3.29704
[1008]	validation_0-rmse:3.39024	validation_1-rmse:3.29704
[1009]	validation_0-rmse:3.39009	validation_1-rmse:3.29701
[1010]	validation_0-rmse:3.39006	validation_1-rmse:3.29702
[1011]	v

[1134]	validation_0-rmse:3.38075	validation_1-rmse:3.29633
[1135]	validation_0-rmse:3.38063	validation_1-rmse:3.29623
[1136]	validation_0-rmse:3.38058	validation_1-rmse:3.29623
[1137]	validation_0-rmse:3.38045	validation_1-rmse:3.29619
[1138]	validation_0-rmse:3.3804	validation_1-rmse:3.29622
[1139]	validation_0-rmse:3.38032	validation_1-rmse:3.29619
[1140]	validation_0-rmse:3.38026	validation_1-rmse:3.2962
[1141]	validation_0-rmse:3.3802	validation_1-rmse:3.29622
[1142]	validation_0-rmse:3.38013	validation_1-rmse:3.2962
[1143]	validation_0-rmse:3.3801	validation_1-rmse:3.29619
[1144]	validation_0-rmse:3.38005	validation_1-rmse:3.29622
[1145]	validation_0-rmse:3.38	validation_1-rmse:3.29624
[1146]	validation_0-rmse:3.37997	validation_1-rmse:3.29625
[1147]	validation_0-rmse:3.37991	validation_1-rmse:3.29626
[1148]	validation_0-rmse:3.37986	validation_1-rmse:3.29623
[1149]	validation_0-rmse:3.37981	validation_1-rmse:3.29623
[1150]	validation_0-rmse:3.37973	validation_1-rmse:3.2962
[1151]

[1274]	validation_0-rmse:3.37118	validation_1-rmse:3.29562
[1275]	validation_0-rmse:3.37117	validation_1-rmse:3.2956
[1276]	validation_0-rmse:3.37111	validation_1-rmse:3.29557
[1277]	validation_0-rmse:3.37107	validation_1-rmse:3.29556
[1278]	validation_0-rmse:3.37102	validation_1-rmse:3.29558
[1279]	validation_0-rmse:3.37093	validation_1-rmse:3.29559
[1280]	validation_0-rmse:3.37087	validation_1-rmse:3.29559
[1281]	validation_0-rmse:3.37075	validation_1-rmse:3.29561
[1282]	validation_0-rmse:3.37068	validation_1-rmse:3.29568
[1283]	validation_0-rmse:3.37056	validation_1-rmse:3.29568
[1284]	validation_0-rmse:3.37042	validation_1-rmse:3.29556
[1285]	validation_0-rmse:3.37029	validation_1-rmse:3.29555
[1286]	validation_0-rmse:3.37018	validation_1-rmse:3.2955
[1287]	validation_0-rmse:3.37013	validation_1-rmse:3.29551
[1288]	validation_0-rmse:3.36999	validation_1-rmse:3.29551
[1289]	validation_0-rmse:3.36987	validation_1-rmse:3.29552
[1290]	validation_0-rmse:3.36983	validation_1-rmse:3.29552

In [33]:
from xgboost import XGBRegressor
clf = XGBRegressor(learning_rate=0.01, max_depth=4, n_estimators=3000, nthread=-1, reg_alpha=1, reg_lambda=1)
drop_cols = [
    'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]

labeled_data, target_col_name = dummy_train[dummy_train.day >= 21].drop(drop_cols, 1), 'log_neg_amount'
max_day = labeled_data.day.max()
c = labeled_data.columns.difference([target_col_name])
metric_by_shift = {}
day_shift = 30 
train_sample = labeled_data[labeled_data.day <= max_day - day_shift]
test_sample = labeled_data[labeled_data.day > max_day - day_shift]

eval_set  = [(train_sample[c], train_sample[target_col_name]), (test_sample[c],test_sample[target_col_name])]
clf.fit(train_sample[c], train_sample[target_col_name], 
        early_stopping_rounds=50, eval_metric='rmse',
        eval_set=eval_set)

predicted_volume = clf.predict(test_sample[c])
metric_by_shift['RMSLE with {} days'.format(day_shift)] = rmsle_by_logs(
    predicted_volume, test_sample[target_col_name])
print(metric_by_shift)

[0]	validation_0-rmse:9.90975	validation_1-rmse:10.2383
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:9.82307	validation_1-rmse:10.1474
[2]	validation_0-rmse:9.73735	validation_1-rmse:10.0573
[3]	validation_0-rmse:9.6526	validation_1-rmse:9.96848
[4]	validation_0-rmse:9.56881	validation_1-rmse:9.88031
[5]	validation_0-rmse:9.48597	validation_1-rmse:9.79348
[6]	validation_0-rmse:9.40405	validation_1-rmse:9.70745
[7]	validation_0-rmse:9.32306	validation_1-rmse:9.62257
[8]	validation_0-rmse:9.24298	validation_1-rmse:9.53857
[9]	validation_0-rmse:9.16381	validation_1-rmse:9.4553
[10]	validation_0-rmse:9.08553	validation_1-rmse:9.37326
[11]	validation_0-rmse:9.00814	validation_1-rmse:9.29203
[12]	validation_0-rmse:8.93163	validation_1-rmse:9.21151
[13]	validation_0-rmse:8.856	validation_1-rmse:9.13191
[14]	validation_0-rmse:8.78123	validation_1-rmse:9.05344


[141]	validation_0-rmse:4.10211	validation_1-rmse:4.07675
[142]	validation_0-rmse:4.08944	validation_1-rmse:4.06316
[143]	validation_0-rmse:4.07697	validation_1-rmse:4.04967
[144]	validation_0-rmse:4.06471	validation_1-rmse:4.03632
[145]	validation_0-rmse:4.05265	validation_1-rmse:4.02322
[146]	validation_0-rmse:4.0408	validation_1-rmse:4.01051
[147]	validation_0-rmse:4.02916	validation_1-rmse:3.99796
[148]	validation_0-rmse:4.01771	validation_1-rmse:3.98557
[149]	validation_0-rmse:4.00645	validation_1-rmse:3.97335
[150]	validation_0-rmse:3.99537	validation_1-rmse:3.96134
[151]	validation_0-rmse:3.98447	validation_1-rmse:3.94959
[152]	validation_0-rmse:3.97376	validation_1-rmse:3.9381
[153]	validation_0-rmse:3.96323	validation_1-rmse:3.92674
[154]	validation_0-rmse:3.95288	validation_1-rmse:3.91556
[155]	validation_0-rmse:3.94269	validation_1-rmse:3.90449
[156]	validation_0-rmse:3.93267	validation_1-rmse:3.8937
[157]	validation_0-rmse:3.92283	validation_1-rmse:3.8831
[158]	validation_0

[283]	validation_0-rmse:3.42842	validation_1-rmse:3.35727
[284]	validation_0-rmse:3.42734	validation_1-rmse:3.35616
[285]	validation_0-rmse:3.42629	validation_1-rmse:3.35519
[286]	validation_0-rmse:3.42525	validation_1-rmse:3.35421
[287]	validation_0-rmse:3.42423	validation_1-rmse:3.35315
[288]	validation_0-rmse:3.42324	validation_1-rmse:3.35221
[289]	validation_0-rmse:3.42226	validation_1-rmse:3.3513
[290]	validation_0-rmse:3.42129	validation_1-rmse:3.35032
[291]	validation_0-rmse:3.42034	validation_1-rmse:3.34943
[292]	validation_0-rmse:3.41941	validation_1-rmse:3.34857
[293]	validation_0-rmse:3.41848	validation_1-rmse:3.34773
[294]	validation_0-rmse:3.41757	validation_1-rmse:3.34691
[295]	validation_0-rmse:3.41668	validation_1-rmse:3.34603
[296]	validation_0-rmse:3.4158	validation_1-rmse:3.3452
[297]	validation_0-rmse:3.41493	validation_1-rmse:3.34447
[298]	validation_0-rmse:3.4141	validation_1-rmse:3.34364
[299]	validation_0-rmse:3.41325	validation_1-rmse:3.34288
[300]	validation_0

[425]	validation_0-rmse:3.36317	validation_1-rmse:3.30568
[426]	validation_0-rmse:3.36295	validation_1-rmse:3.30562
[427]	validation_0-rmse:3.36273	validation_1-rmse:3.30552
[428]	validation_0-rmse:3.36253	validation_1-rmse:3.30539
[429]	validation_0-rmse:3.36235	validation_1-rmse:3.30535
[430]	validation_0-rmse:3.36214	validation_1-rmse:3.30524
[431]	validation_0-rmse:3.36195	validation_1-rmse:3.30518
[432]	validation_0-rmse:3.36175	validation_1-rmse:3.30509
[433]	validation_0-rmse:3.36154	validation_1-rmse:3.30505
[434]	validation_0-rmse:3.36134	validation_1-rmse:3.30496
[435]	validation_0-rmse:3.36116	validation_1-rmse:3.30486
[436]	validation_0-rmse:3.36099	validation_1-rmse:3.30469
[437]	validation_0-rmse:3.3608	validation_1-rmse:3.30457
[438]	validation_0-rmse:3.36058	validation_1-rmse:3.30453
[439]	validation_0-rmse:3.36039	validation_1-rmse:3.30445
[440]	validation_0-rmse:3.3602	validation_1-rmse:3.30434
[441]	validation_0-rmse:3.36003	validation_1-rmse:3.30426
[442]	validation

[567]	validation_0-rmse:3.34067	validation_1-rmse:3.29953
[568]	validation_0-rmse:3.34052	validation_1-rmse:3.29947
[569]	validation_0-rmse:3.34044	validation_1-rmse:3.29946
[570]	validation_0-rmse:3.34035	validation_1-rmse:3.29942
[571]	validation_0-rmse:3.34027	validation_1-rmse:3.29942
[572]	validation_0-rmse:3.34012	validation_1-rmse:3.29945
[573]	validation_0-rmse:3.33999	validation_1-rmse:3.29936
[574]	validation_0-rmse:3.33991	validation_1-rmse:3.29935
[575]	validation_0-rmse:3.33982	validation_1-rmse:3.29928
[576]	validation_0-rmse:3.33966	validation_1-rmse:3.29926
[577]	validation_0-rmse:3.33957	validation_1-rmse:3.29926
[578]	validation_0-rmse:3.33947	validation_1-rmse:3.29925
[579]	validation_0-rmse:3.33938	validation_1-rmse:3.29924
[580]	validation_0-rmse:3.3392	validation_1-rmse:3.2992
[581]	validation_0-rmse:3.33911	validation_1-rmse:3.29919
[582]	validation_0-rmse:3.33898	validation_1-rmse:3.29917
[583]	validation_0-rmse:3.3389	validation_1-rmse:3.29911
[584]	validation_

[709]	validation_0-rmse:3.32642	validation_1-rmse:3.2973
[710]	validation_0-rmse:3.32633	validation_1-rmse:3.29726
[711]	validation_0-rmse:3.32619	validation_1-rmse:3.29725
[712]	validation_0-rmse:3.32608	validation_1-rmse:3.2972
[713]	validation_0-rmse:3.32599	validation_1-rmse:3.2972
[714]	validation_0-rmse:3.32588	validation_1-rmse:3.29718
[715]	validation_0-rmse:3.32572	validation_1-rmse:3.29721
[716]	validation_0-rmse:3.32556	validation_1-rmse:3.29723
[717]	validation_0-rmse:3.32542	validation_1-rmse:3.29721
[718]	validation_0-rmse:3.32535	validation_1-rmse:3.2972
[719]	validation_0-rmse:3.32524	validation_1-rmse:3.2973
[720]	validation_0-rmse:3.32517	validation_1-rmse:3.29727
[721]	validation_0-rmse:3.32505	validation_1-rmse:3.29725
[722]	validation_0-rmse:3.32495	validation_1-rmse:3.29724
[723]	validation_0-rmse:3.32486	validation_1-rmse:3.29718
[724]	validation_0-rmse:3.32477	validation_1-rmse:3.29715
[725]	validation_0-rmse:3.32463	validation_1-rmse:3.2971
[726]	validation_0-r

In [ ]:
from xgboost import XGBRegressor
clf = XGBRegressor(min_child_weight=
    learning_rate=0.05, max_depth=5, n_estimators=3000,  
    reg_alpha=1, reg_lambda=1, nthread=-1
)
drop_cols = [
    'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]

labeled_data, target_col_name = dummy_train[dummy_train.day >= 21].drop(drop_cols, 1), 'log_neg_amount'
max_day = labeled_data.day.max()
c = labeled_data.columns.difference([target_col_name])
metric_by_shift = {}
day_shift = 30 
train_sample = labeled_data[labeled_data.day <= max_day - day_shift]
test_sample = labeled_data[labeled_data.day > max_day - day_shift]

eval_set  = [(train_sample[c], train_sample[target_col_name]), (test_sample[c],test_sample[target_col_name])]
clf.fit(train_sample[c], train_sample[target_col_name], 
        early_stopping_rounds=50, eval_metric='rmse',
        eval_set=eval_set)

predicted_volume = clf.predict(test_sample[c])
metric_by_shift['RMSLE with {} days'.format(day_shift)] = rmsle_by_logs(
    predicted_volume, test_sample[target_col_name])
print(metric_by_shift)

In [31]:
clf = XGBRegressor(learning_rate=0.01, max_depth=4, n_estimators=1308, nthread=-1, reg_alpha=1, reg_lambda=1)

drop_cols = [
    #'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]
    
submit_table = get_agile_prediction(
    dummy_train, 
    dummy_train.log_neg_amount, 
    dummy_test.drop(drop_cols, 1), 
    clf, test_transactions)
submit_table.volume = submit_table.volume.apply(lambda x: 0 if x<0 else x)
submit_table.to_csv('agile_n_trans_xgb_neg-1300.csv', index=False)
submit_table.head()

100%|██████████| 5/5 [17:24<00:00, 207.85s/it]


,id,volume
0,4814-457,8054894.5
1,4814-458,10279853.0
2,4814-459,14983682.0
3,4814-460,10592767.0
4,4814-461,10561537.0


In [60]:
clf = XGBRegressor(learning_rate=0.01, max_depth=4, n_estimators=5008, nthread=-1, reg_alpha=1, reg_lambda=1)

drop_cols = [
    #'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]
    
submit_table = get_agile_prediction(
    dummy_train, 
    dummy_train.log_neg_amount, 
    dummy_test.drop(drop_cols, 1), 
    clf, test_transactions)
submit_table.volume = submit_table.volume.apply(lambda x: 0 if x<0 else x)
submit_table.to_csv('agile_n_trans_xgb_neg-5000.csv', index=False)
submit_table.head()


  0%|          | 0/5 [00:00<?, ?it/s]

[0]	validation_0-rmse:9.89491	validation_1-rmse:10.2385
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:9.80901	validation_1-rmse:10.1475
[2]	validation_0-rmse:9.72407	validation_1-rmse:10.0575
[3]	validation_0-rmse:9.6401	validation_1-rmse:9.96856
[4]	validation_0-rmse:9.55709	validation_1-rmse:9.88073
[5]	validation_0-rmse:9.47499	validation_1-rmse:9.79386
[6]	validation_0-rmse:9.39386	validation_1-rmse:9.70791
[7]	validation_0-rmse:9.31363	validation_1-rmse:9.62297
[8]	validation_0-rmse:9.2343	validation_1-rmse:9.53897
[9]	validation_0-rmse:9.1559	validation_1-rmse:9.45617
[10]	validation_0-rmse:9.0784	validation_1-rmse:9.37401
[11]	validation_0-rmse:9.00177	validation_1-rmse:9.29281
[12]	validation_0-rmse:8.92604	validation_1-rmse:9.21254
[13]	validation_0-rmse:8.85118	validation_1-rmse:9.13339
[14]	validation_0-rmse:8.77717	validation_1-rmse:9.05478

[38]	validation_0-rmse:7.24012	validation_1-rmse:7.41872
[39]	validation_0-rmse:7.18514	validation_1-rmse:7.35971
[40]	validation_0-rmse:7.13093	validation_1-rmse:7.30188
[41]	validation_0-rmse:7.07737	validation_1-rmse:7.24455
[42]	validation_0-rmse:7.0244	validation_1-rmse:7.1876
[43]	validation_0-rmse:6.97213	validation_1-rmse:7.13147
[44]	validation_0-rmse:6.92051	validation_1-rmse:7.0763
[45]	validation_0-rmse:6.86953	validation_1-rmse:7.02147
[46]	validation_0-rmse:6.81917	validation_1-rmse:6.96732
[47]	validation_0-rmse:6.76948	validation_1-rmse:6.91438
[48]	validation_0-rmse:6.72038	validation_1-rmse:6.86161
[49]	validation_0-rmse:6.6719	validation_1-rmse:6.8096
[50]	validation_0-rmse:6.62406	validation_1-rmse:6.75812
[51]	validation_0-rmse:6.5768	validation_1-rmse:6.70731
[52]	validation_0-rmse:6.53014	validation_1-rmse:6.65716
[53]	validation_0-rmse:6.48406	validation_1-rmse:6.60752
[54]	validation_0-rmse:6.43859	validation_1-rmse:6.5585
[55]	validation_0-rmse:6.3937	validati

[182]	validation_0-rmse:3.79849	validation_1-rmse:3.67109
[183]	validation_0-rmse:3.79238	validation_1-rmse:3.66432
[184]	validation_0-rmse:3.78631	validation_1-rmse:3.65769
[185]	validation_0-rmse:3.78041	validation_1-rmse:3.6511
[186]	validation_0-rmse:3.77459	validation_1-rmse:3.64477
[187]	validation_0-rmse:3.76891	validation_1-rmse:3.63849
[188]	validation_0-rmse:3.76329	validation_1-rmse:3.63221
[189]	validation_0-rmse:3.75777	validation_1-rmse:3.62627
[190]	validation_0-rmse:3.75238	validation_1-rmse:3.6203
[191]	validation_0-rmse:3.74709	validation_1-rmse:3.61432
[192]	validation_0-rmse:3.74187	validation_1-rmse:3.60869
[193]	validation_0-rmse:3.73678	validation_1-rmse:3.60304
[194]	validation_0-rmse:3.73172	validation_1-rmse:3.59762
[195]	validation_0-rmse:3.72681	validation_1-rmse:3.5922
[196]	validation_0-rmse:3.72195	validation_1-rmse:3.58678
[197]	validation_0-rmse:3.71718	validation_1-rmse:3.58161
[198]	validation_0-rmse:3.71248	validation_1-rmse:3.57644
[199]	validation_

[324]	validation_0-rmse:3.47639	validation_1-rmse:3.32959
[325]	validation_0-rmse:3.47585	validation_1-rmse:3.3291
[326]	validation_0-rmse:3.47527	validation_1-rmse:3.32863
[327]	validation_0-rmse:3.47472	validation_1-rmse:3.32822
[328]	validation_0-rmse:3.47416	validation_1-rmse:3.32783
[329]	validation_0-rmse:3.47364	validation_1-rmse:3.32745
[330]	validation_0-rmse:3.4731	validation_1-rmse:3.32701
[331]	validation_0-rmse:3.47256	validation_1-rmse:3.3265
[332]	validation_0-rmse:3.47204	validation_1-rmse:3.32615
[333]	validation_0-rmse:3.47152	validation_1-rmse:3.32578
[334]	validation_0-rmse:3.47101	validation_1-rmse:3.32537
[335]	validation_0-rmse:3.47051	validation_1-rmse:3.32503
[336]	validation_0-rmse:3.47001	validation_1-rmse:3.32456
[337]	validation_0-rmse:3.46953	validation_1-rmse:3.32426
[338]	validation_0-rmse:3.46906	validation_1-rmse:3.32396
[339]	validation_0-rmse:3.46862	validation_1-rmse:3.32362
[340]	validation_0-rmse:3.46817	validation_1-rmse:3.32327
[341]	validation_

...

[709]	validation_0-rmse:3.64287	validation_1-rmse:3.36294
[710]	validation_0-rmse:3.64272	validation_1-rmse:3.36292
[711]	validation_0-rmse:3.64262	validation_1-rmse:3.36292
[712]	validation_0-rmse:3.64251	validation_1-rmse:3.36287
[713]	validation_0-rmse:3.64234	validation_1-rmse:3.36272
[714]	validation_0-rmse:3.64223	validation_1-rmse:3.36273
[715]	validation_0-rmse:3.64211	validation_1-rmse:3.36269
[716]	validation_0-rmse:3.642	validation_1-rmse:3.36263
[717]	validation_0-rmse:3.64185	validation_1-rmse:3.36257
[718]	validation_0-rmse:3.64172	validation_1-rmse:3.36256
[719]	validation_0-rmse:3.6416	validation_1-rmse:3.3626
[720]	validation_0-rmse:3.64145	validation_1-rmse:3.36253
[721]	validation_0-rmse:3.6413	validation_1-rmse:3.3625
[722]	validation_0-rmse:3.64117	validation_1-rmse:3.36249
[723]	validation_0-rmse:3.64099	validation_1-rmse:3.36244
[724]	validation_0-rmse:3.64086	validation_1-rmse:3.36239
[725]	validation_0-rmse:3.64075	validation_1-rmse:3.36237
[726]	validation_0-r

[851]	validation_0-rmse:3.62538	validation_1-rmse:3.36103
[852]	validation_0-rmse:3.62528	validation_1-rmse:3.36101
[853]	validation_0-rmse:3.62514	validation_1-rmse:3.36103
[854]	validation_0-rmse:3.62501	validation_1-rmse:3.36105
[855]	validation_0-rmse:3.62492	validation_1-rmse:3.36104
[856]	validation_0-rmse:3.62479	validation_1-rmse:3.36106
[857]	validation_0-rmse:3.62469	validation_1-rmse:3.36103
[858]	validation_0-rmse:3.62458	validation_1-rmse:3.36099
[859]	validation_0-rmse:3.62449	validation_1-rmse:3.36091
[860]	validation_0-rmse:3.62439	validation_1-rmse:3.36092
[861]	validation_0-rmse:3.62427	validation_1-rmse:3.36091
[862]	validation_0-rmse:3.62417	validation_1-rmse:3.3609
[863]	validation_0-rmse:3.62406	validation_1-rmse:3.36091
[864]	validation_0-rmse:3.62391	validation_1-rmse:3.36095
[865]	validation_0-rmse:3.62378	validation_1-rmse:3.36088
[866]	validation_0-rmse:3.62368	validation_1-rmse:3.36087
[867]	validation_0-rmse:3.62357	validation_1-rmse:3.36087
[868]	validatio

[993]	validation_0-rmse:3.60963	validation_1-rmse:3.35945
[994]	validation_0-rmse:3.60953	validation_1-rmse:3.35945
[995]	validation_0-rmse:3.60944	validation_1-rmse:3.35942
[996]	validation_0-rmse:3.60931	validation_1-rmse:3.35939
[997]	validation_0-rmse:3.60917	validation_1-rmse:3.35935
[998]	validation_0-rmse:3.60907	validation_1-rmse:3.35932
[999]	validation_0-rmse:3.60898	validation_1-rmse:3.35933
[1000]	validation_0-rmse:3.60889	validation_1-rmse:3.35928
[1001]	validation_0-rmse:3.60882	validation_1-rmse:3.35926
[1002]	validation_0-rmse:3.6087	validation_1-rmse:3.35925
[1003]	validation_0-rmse:3.60863	validation_1-rmse:3.35926
[1004]	validation_0-rmse:3.60851	validation_1-rmse:3.35948
[1005]	validation_0-rmse:3.60838	validation_1-rmse:3.35951
[1006]	validation_0-rmse:3.6083	validation_1-rmse:3.3595
[1007]	validation_0-rmse:3.6082	validation_1-rmse:3.35948
[1008]	validation_0-rmse:3.60811	validation_1-rmse:3.35949
[1009]	validation_0-rmse:3.60799	validation_1-rmse:3.35946
[1010]	v

100%|██████████| 5/5 [24:05<00:00, 305.98s/it]


,id,volume
0,4814-457,8054894.5
1,4814-458,10279853.0
2,4814-459,14983682.0
3,4814-460,10592767.0
4,4814-461,10561537.0


In [27]:

rmsle_by_logs(
    clf.predict(test_sample[c], clf.best_iteration), test_sample[target_col_name])

3.3041410690649133

In [ ]:
clf = XGBRegressor(learning_rate=0.02, max_depth=6, n_estimators=350, nthread=-1, reg_alpha=1, reg_lambda=1)

drop_cols = [
    #'log_pos_amount', 'n_transactions', 'neg_amount', 'pos_amount'
]
for prev_week_index in range(26,35):
    drop_cols += ['prev_week_{}_neg'.format(prev_week_index),
                'prev_week_{}_mean_neg'.format(prev_week_index),
                'prev_week_{}_std_neg'.format(prev_week_index)]

for prev_month_index in []:
    drop_cols += ['prev_month_1_mean_neg'.format(prev_month_index), 
                  'prev_month_1_std_neg'.format(prev_month_index),
                  'prev_month_1_median_neg'.format(prev_month_index),
                  'prev_month_1_max_neg'.format(prev_month_index),
                  'prev_month_1_min_neg'.format(prev_month_index),
                 ]
    
submit_table = get_agile_prediction(
    dummy_train, 
    dummy_train.log_neg_amount, 
    dummy_test.drop(drop_cols, 1), 
    clf, test_transactions)
submit_table.volume = submit_table.volume.apply(lambda x: 0 if x<0 else x)
submit_table.to_csv('agile_n_trans_xgb_neg.csv', index=False)
submit_table.head()